In [1]:
import h5py
import numpy as np
import math
import glob
import os
import umap
import hdbscan
import seaborn as sns
import pandas as pd
import pickle
import openpyxl
import cv2
from scipy import ndimage as ndi

from functools import reduce
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.optimize import curve_fit
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
import sklearn.cluster as cluster
from moviepy.editor import *
import imageio


/Users/kaavya/anaconda3/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/kaavya/anaconda3/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/kaavya/anaconda3/lib/python3.11/site-packages/umap/distances.py:1086: NumbaDeprecatio

# General Functions

In [2]:
def get_running_total(lst: list) -> list:
    """
    Compute the running total for a list of numbers.
    
    Given a list of numbers, this function returns a new list 
    where each element is the cumulative sum of the elements 
    up to that point in the original list.

    Parameters:
    - lst (list): A list of numbers.

    Returns:
    - list: A list containing the running total.

    Example:
    >>> get_running_total([1, 2, 3, 4])
    [1, 3, 6, 10]
    """

    def accumulator(acc, x):
        if acc:
            return acc + [acc[-1] + x]
        else:
            return [x]
        
    return reduce(accumulator, lst, [])

def make_dict(my_list):
    my_dict = {}
    for i in range(len(my_list)):
        my_dict[my_list[i]] = i
    return my_dict

def fill_missing(Y, kind="linear"):
    """Fills missing values independently along each dimension after the first."""

    # Store initial shape.
    initial_shape = Y.shape

    # Flatten after first dim.
    Y = Y.reshape((initial_shape[0], -1))

    # Interpolate along each slice.
    for i in range(Y.shape[-1]):
        y = Y[:, i]

        # Build interpolant.
        x = np.flatnonzero(~np.isnan(y))
        f = interp1d(x, y[x], kind=kind, fill_value=np.nan, bounds_error=False)

        # Fill missing
        xq = np.flatnonzero(np.isnan(y))
        y[xq] = f(xq)
        
        # Fill leading or trailing NaNs with the nearest non-NaN values
        mask = np.isnan(y)
        y[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), y[~mask])

        # Save slice
        Y[:, i] = y

    # Restore to initial shape.
    Y = Y.reshape(initial_shape)

    return Y

def smooth_diff(node_loc, deriv, win=25, poly=3):
    """
    node_loc is a [frames, 2] array
    
    win defines the window to smooth over
    
    poly defines the order of the polynomial
    to fit with
    
    """
    node_vel = np.zeros_like(node_loc)
    for c in range(node_loc.shape[-1]):
        node_vel[:, c] = savgol_filter(node_loc[:, c], win, poly, deriv)
    if deriv != 0:
        node_vel = np.linalg.norm(node_vel,axis=1)
    return node_vel

class sleap_vid():

    def __init__(self, filename, track_no):
        self.name = filename
        self.features = {}
        self.mice = track_no
        self.get_info(filename)

    def get_info(self, filename):
        with h5py.File(filename, "r") as f:
            tracks= [n.decode() for n in f["track_names"][:]]
            locations = f["tracks"][:].T
            locations = locations[:,:,:,:self.mice]
            self.locations = fill_missing(locations)
            nodes = [n.decode() for n in f["node_names"][:]]
            self.tracks = tracks
            self.nodes = nodes
            self.track_dict = make_dict(tracks)
            self.node_dict = make_dict(nodes)

    def smooth_locations(self, win=25, poly=3):
        for node in self.node_dict.values():
            for track in range(self.mice):
                nodeloc = self.locations[:,node,:,track]
                smoothed_node = smooth_diff(nodeloc, deriv=0, win=win, poly=poly)
                self.locations[:,node,:,track] = smoothed_node
        

    def node_velocity(self, node, win=25, poly=3, normalization_factor=None):
        """
        takes in node and returns the velocity of that node 
        for each mouse

        Args: 
            node: string, name of node
        
        Returns:
            velocities: 2d np.array of floats (d = 2 x #of frames)
                where each element is the velocity for that node
                distances[0] = velocities for mouse2
                distances[1] = velocities for mouse2
        """
        node_loc1 = self.locations[:, self.node_dict[node], :, 0]
        node_loc2 = self.locations[:, self.node_dict[node], :, 1]
        if normalization_factor != None:
            node_loc1 = node_loc1 * normalization_factor
            node_loc2 = node_loc2 * normalization_factor
        m1_vel = smooth_diff(node_loc1, deriv = 1, win=win, poly=poly)
        m2_vel = smooth_diff(node_loc2, deriv = 1, win=win, poly=poly)
        velocities = np.array([m1_vel,m2_vel])
        return velocities 
   
    def distances_between_mice(self, node, normalization_factor=None):
        """
        takes in node name
        returns a list of distances between the nodes of the two mice

        Args:
            node: string, name of node
        Returns:
            c_list: 1D np.array of floats (d = # of frames)
        """
        locations = self.locations 
        node_index = self.node_dict[node]
        x1 = locations[:,node_index,0,0]
        y1 = locations[:,node_index,1,0]
        # x , y coordinate of nose for mouse 1
        x2 = locations[:,node_index,0,1]
        y2 =  locations[:,node_index,1,1]
        # x and y coordinate of nose of mouse 2
        # solve for c using pythagroean theory
        distances = np.sqrt(((x1 - x2)**2) + ((y1 - y2)**2))
        if normalization_factor != None: 
            distances = distances * normalization_factor
            distances = distances.T.flatten()
        return distances
    
    def distances_between_nodes(self, node1, node2, normalization_factor = None):
        """
        takes in two nodes and returns the distances between those nodes 
        for each mouse

        Args: 
            node1: string, name of node 1
            node2: string, name of node 2 
        
        Returns:
            distances: 2d np.array of floats (d = 2 x #of frames)
                where each element is the distance between node1 and node2 
                distances[0] = distances for mouse1
                distances[1] = distances for mouse2
        """
        locations = self.locations
        node_index1 = self.node_dict[node1]
        node_index2 = self.node_dict[node2]
        x1,y1 = locations[:, node_index1,0,0], locations[:,node_index1,1,0]
        # x , y coordinate of node 1 for mouse 1
        x2,y2 = locations[:,node_index2,0,0], locations[:,node_index2,1,0]
        # x, y coordiantes of node 2 for mouse 1
        x3, y3 = locations[:,node_index1,0,1], locations[:,node_index1,1,1]
        # x and y coordinate of node 1 of mouse 2
        x4, y4 = locations[:,node_index2,0,1], locations[:,node_index2,1,1]
        # solve for c using pythagroean theory
        c2 = np.sqrt(((x3 -x4)**2)+ ((y3 - y4)**2))
        c1 = np.sqrt(((x1 - x2)**2) + ((y1 - y2)**2))
        if normalization_factor != None:
            c2 = (c2*normalization_factor).T.flatten()
            c1 = (c1*normalization_factor).T.flatten()
        distances = np.array[c1, c2]
        return distances
    
    def distances_to_point(self, node, point, normalization_factor = None):
        # CONFIRMED THAT IT WORKS in terms of doing the math by hand
        """
        takes in two nodes and returns the distances between those nodes 
        for each mouse

        Args: 
            node1: string, name of node 1
            point: tuple or list of two floats,
                x and y coordinate of desired point

        Returns:
            distances: 2d np.array of floats (d = 2 x #of frames)
                where each element is the distance between node and point 
                distances[0] = distances for mouse1
                distances[1] = distances for mouse2
        """
        locations = self.locations
        node_index = self.node_dict[node]
        px, py = point[0], point[1]
        x1,y1 = locations[:, node_index,0,0], locations[:,node_index,1,0]
        # x , y coordinate of node 1 for mouse 1
        # x, y coordiantes of node 2 for mouse 1
        x3, y3 = locations[:,node_index,0,1], locations[:,node_index,1,1]
        # x and y coordinate of node 1 of mouse 2
        px = np.full_like(x1, fill_value=px)
        py = np.full_like(y1, fill_value=py)
        # solve for c using pythagroean theory
        c2 = np.sqrt(((x3 -px)**2)+ ((y3 - py)**2))
        c1 = np.sqrt(((x1 - px)**2) + ((y1 - py)**2))
        if normalization_factor != None:
            c2 = (c2*normalization_factor).T.flatten()
            c1 = (c1*normalization_factor).T.flatten()
        distances = np.array([c1, c2])
        return distances


    def node_angles(self, node1, node2, node3):
        """
        takes in locations and three nodes, calculates angle between the three points 
        with the second node being the center point
        
        Args:  
            node1: string, name of node 1
            node2: string, name of node 2 
            node3: string, name of node 3

        Returns:
            ang: 2d np. array (d = 2 x # of frames)
                where each element is the angle between 
                node1 and node3 with node2 as center point 
                anles_all_mice[0] = angles for mouse1
                anles_all_mice[1] = angles for mouse2
        """
        locations = self.locations  
        node_index_1 = self.node_dict[node1]
        node_index_2 = self.node_dict[node2]
        node_index_3 = self.node_dict[node3]
        ax = locations[:,node_index_1, 0, :]
        ay = locations[:,node_index_1, 1, :]
        bx = locations[:,node_index_2, 0, :]
        by = locations[:,node_index_2, 1, :]
        cx = locations[:,node_index_3,0,:]
        cy = locations[:,node_index_3, 1, :]
        ang = np.arctan2(cy-by, cx-bx) - np.arctan2(ay-by, ax-bx) 
        ang_swapped = np.arctan2(cy-by, cx-bx) - np.arctan2(cy-by, cx-bx) 
        ang = np.maximum(ang, ang_swapped)
        return ang.T
    
    def point_angles(self, node1, node2, point):
        """
        takes in locations and three nodes, calculates angle between the three points 
        with the second node being the center point
        
        Args:  
            node1: string, name of node 1
            node2: string, name of node 2 
            point: tuple or list of two floats,
                x and y coordinate of desired point

        Returns:
            ang: 2d np. array (d = 2 x # of frames)
                where each element is the angle between 
                node1 and point with node2 as center point 
                anles_all_mice[0] = angles for mouse1
                anles_all_mice[1] = angles for mouse2
        """
        locations = self.locations  
        node_index_1 = self.node_dict[node1]
        node_index_2 = self.node_dict[node2]
        px, py = point[0], point[1]
        ax = locations[:,node_index_1, 0, :]
        ay = locations[:,node_index_1, 1, :]
        bx = locations[:,node_index_2, 0, :]
        by = locations[:,node_index_2, 1, :]
        px = np.full_like(ax, fill_value=px)
        py = np.full_like(ay, fill_value=py)
        ang = np.arctan2(py-by, px-bx) - np.arctan2(ay-by, ax-bx) 
        ang_swapped = np.arctan2(ay-by, ax-bx) - np.arctan2(py-by, px-bx) 
        ang = np.maximum(ang, ang_swapped)
        return ang.T

    def orientation(self):
        """
        Takes in locations and nose and thorax node index to calculate the angle of orientation
        between mice where two mice facing each other results in pi
        theta = 0 means they are not facing each other 
        
        """
        locations = self.locations
        nose_node = self.node_dict['nose']
        thorax_node = self.node_dict['thorax']
        forehead = self.node_dict['forehead']
        ax = locations[:, nose_node, 0, 0]
        ay = locations[:, nose_node, 1, 0]
        bx = locations[:,thorax_node, 0, 0]
        by = locations[:,thorax_node, 1, 0]
        cx = locations[:, forehead, 0, 1]
        cy = locations[:,forehead, 1, 1]
        ang_m1 = np.arctan2(cy-by, cx-bx) - np.arctan2(ay-by, ax-bx) 
        ang_m1_swapped = np.arctan2(ay-by, ax-bx) - np.arctan2(cy-by, cx-bx)
        ax = locations[:, nose_node, 0, 1]
        ay = locations[:, nose_node, 1, 1]
        bx = locations[:,thorax_node, 0, 1]
        by = locations[:,thorax_node, 1, 1]
        cx = locations[:, forehead, 0, 0]
        cy = locations[:,forehead, 1, 0]
        ang_m2 = np.arctan2(cy-by, cx-bx) - np.arctan2(ay-by, ax-bx) 
        ang_m2_swapped = np.arctan2(ay-by, ax-bx) - np.arctan2(cy-by, cx-bx) 
        ang_m1 = np.maximum(ang_m1,ang_m1_swapped)
        ang_m2 = np.maximum(ang_m2,ang_m2_swapped)
        return np.array([ang_m1, ang_m2])
    
    def create_events_array(thresh, feature, greater_than = False):
        """
        This function will create a 2D array of start and stop frames for which 
        a given feature passes a given threshold. The output is an array of two
        element arrays, where the first value is the start frame and the second 
        value is the stop frame. 

        Args (3 total, 2 required):
            thresh: float, threshold value to filter frames
            feature: 1D numpy array, values of some feature for each frame
            greater_than: Bool, default = False, True if finding frames where feature >= thresh, False for <=

        Return (1):
            output: 2D numpy array, Pairs of start and stop frames for which the value of the feature agrees with your filter    
        """
        if greater_than:
            ind1 = feature.index[feature >= thresh].tolist()
        else:
            ind1 = feature.index[feature <= thresh].tolist()
        start = []
        start.append(ind1[0])

        end = []

        for i in range(len(ind1)-1):
            if (ind1[i]-ind1[i-1])>3:
                start.append(ind1[i])
            
            if (ind1[i+1]-ind1[i])>3:
                end.append(ind1[i])

        end.append(ind1[-1])

        output = np.column_stack((start, end))
        return output
    
    
"""""
    def event_maker(df, event_length, cluster_purity):
        events = []
        #if kmeans is part of the df 
        while i < len(df):
            #checks to see if it is the same video recording
            same_recording = df['Recording'][i] == df['Recording'][i + 1]
            #find out how many frame indices are there, then count
            frame_indice_count++
            
            #check if kmeans is the same or not for each frame indice
            #needs to record start time
            #every x amount of indices you want to look at, find the amount of cluster assignments from the largest category 
            cluster_count = 0 
            cluster_count++
            #find ratio of count to indices 
            cluster_average = cluster_count/frame_indice_count
            if cluster_purity <= cluster_average
                events.append(frame_indice[0], frame_indice[i])
                
"""        
    


'""\n    def event_maker(df, event_length, cluster_purity):\n        events = []\n        #if kmeans is part of the df \n        while i < len(df):\n            #checks to see if it is the same video recording\n            same_recording = df[\'Recording\'][i] == df[\'Recording\'][i + 1]\n            #find out how many frame indices are there, then count\n            frame_indice_count++\n            \n            #check if kmeans is the same or not for each frame indice\n            #needs to record start time\n            #every x amount of indices you want to look at, find the amount of cluster assignments from the largest category \n            cluster_count = 0 \n            cluster_count++\n            #find ratio of count to indices \n            cluster_average = cluster_count/frame_indice_count\n            if cluster_purity <= cluster_average\n                events.append(frame_indice[0], frame_indice[i])\n                \n'

## Tube Test Specific Functions

In [3]:
def get_mouse_sides(locations):
    '''
    takes in locations
    returns (left mouse index, right mouse index)
    '''
    if locations[0,1,0,0] > locations[0,1,0,1]:
        #if mouse 0 has a greater starting x value for its nose, then it is the right mouse
        return (1, 0)
    else:
        return (0, 1)
    
def smooth_diff_one_dim(node_loc, win=25, poly=3):
    """
    node_loc is a [frames] array
    
    win defines the window to smooth over
    
    poly defines the order of the polynomial
    to fit with
    
    """
    node_loc_vel = np.zeros_like(node_loc)
    node_loc_vel[:] = savgol_filter(node_loc[:], win, poly, deriv=1)
    return node_loc_vel

def get_velocities(locations, node_index):

    left_mouse, right_mouse = get_mouse_sides(locations)
    node_loc_left = locations[:,node_index, 0,left_mouse]
    # node loc (x,y) of node of mouse 1
    node_loc_right = (locations[:,node_index, 0,right_mouse]) * (-1)
    # x,y's of node of mouse 2 
    
    m1_vel = smooth_diff_one_dim(node_loc_left)
    m2_vel = smooth_diff_one_dim(node_loc_right)
    velocities = [m1_vel,m2_vel]
    return velocities 

def get_distances_between_mice(self, node):
        #confirmed does what i want it to
        """
        takes in locations and node index 
        returns a list of distances between the nodes of the two mice
        """
        c_list = []
        locations = self.locations 
        node_index = node_dict[node]
        left_mouse, right_mouse = get_mouse_sides(locations)
        for i in range(len(locations)):
        
            (x1,y1) = (locations[i,node_index,0,left_mouse], locations[i,node_index,1,left_mouse])
            # x , y coordinate of nose for mouse 1
            (x2, y2) = (locations[i,node_index,0,right_mouse], locations[i,node_index,1,right_mouse])
            # x and y coordinate of nose of mouse 2
            # solve for c using pythagroean theory
            a2 = (x1 - x2) ** 2 
            b2 = (y1 - y2) ** 2 
            c = math.sqrt(a2 + b2)
            if x1 > x2:
                c_list.append(-1*c)
            else:
                c_list.append(c)
        return c_list

# Data Load for Reward Comp


### Box Set Up: reward point coordinate and length of box

In [4]:
path = "/Users/kaavya/Documents/GitHub/social_dominance_active_inference/data/pilot_3/reward_competition/corner_labeling"
ending = "*.h5"
search_path = os.path.join(path, ending)
box_setup ={}
for filename in glob.glob(search_path):
    file = os.path.basename(filename)
    file = file[:file.index('.')]
    box = sleap_vid(filename, track_no=5)
    box_setup[file] = {}
    reward_port = box.locations[:,box.node_dict['reward_port'], :,:].flatten()
    #grab reward port coordinates
    box_setup[file]['reward_port'] = reward_port
    x_diff = box.locations[:,box.node_dict['box_bottom_left'], 0,:] - box.locations[:,box.node_dict['box_bottom_right'], 0,:]
    y_diff = box.locations[:,box.node_dict['box_bottom_left'], 1,:] - box.locations[:,box.node_dict['box_bottom_right'], 1,:]
    box_width = np.sqrt(x_diff**2+y_diff**2)
    box_setup[file]['bottom_left'] = box.locations[:,box.node_dict['box_bottom_left'], :,:].flatten()
    box_setup[file]['bottom_right'] = box.locations[:,box.node_dict['box_bottom_right'],:,:].flatten()
    #create a normalization factor for distances across recordings
    distance_normalization_factor = 300/box_width
    box_setup[file]['distance_normalization_factor'] = distance_normalization_factor



10-04-22_Test_13_5-1v5-3
10-03-22_Test_11_6-1v6-2
10-04-22_Test_21_1-1v1-3
10-04-22_Test_17_3-1v3-3
10-04-22_Test_19_4-1v4-3
10-03-22_Test_12_6-3v6-4
10-04-22_Test_15_6-1v6-3
10-04-22_Test_20_4-2v4-4
10-04-22_Test_16_6-2v6-4
10-04-22_Test_14_5-2v5-4
10-04-22_Test_3_5-1v5-4
10-03-22_Test_35_6-2v6-3
10-03-22_Test_16_1-1v1-3
10-04-22_Test_8_3-2v3-3
10-03-22_Test_32_4-1v4-4
10-03-22_Test_24_5-1v5-3
10-04-22_Test_11_1-1v1-4
10-03-22_Test_14_2-1v2-3
10-04-22_Test_7_3-1v3-4
10-04-22_Test_9_2-1v2-4
10-03-22_Test_20_3-1v3-3
10-03-22_Test_8_4-3v4-4
10-04-22_Test_2_6-2v6-3
10-03-22_Test_1_1-1v1-2
10-04-22_Test_12_1-2v1-3
10-03-22_Test_19_3-2v3-4
10-03-22_Test_3_2-1v2-2
10-04-22_Test_10_2-2v2-3
10-03-22_Test_15_1-2v1-4
10-03-22_Test_13_2-2v2-4
10-03-22_Test_23_5-2v5-4
10-03-22_Test_7_4-1v4-2
10-03-22_Test_2_1-3v1-4


In [5]:
ranking_df = pd.read_excel('/Users/kaavya/Downloads/combined_final_elo_rating_df.xlsx')
filtered_ranking_df = ranking_df[ranking_df['cohort'] == 'pilot_3']
filtered_ranking_df

,Unnamed: 0,cage,cage_num_of_subject,cohort,home_cage_observation_final_elo_rating,home_cage_observation_rank,reward_competition_final_elo_rating,reward_competition_rank,strain,subject_id,tube_test_final_elo_rating,tube_test_rank,urine_marking_final_elo_rating,urine_marking_rank
40,40,NaN,1,pilot_3,1000.0,NaN,1163.2,1,C57,1.1,1165.3,1,1009.7,2
41,41,NaN,1,pilot_3,1000.0,NaN,959.9,3,C57,1.2,1054.5,2,1019.1,1
42,42,NaN,1,pilot_3,1000.0,NaN,1018.4,2,C57,1.3,882.3,4,980.9,4
43,43,NaN,1,pilot_3,1000.0,NaN,858.5,4,C57,1.4,897.9,3,990.3,3
44,44,NaN,2,pilot_3,1169.1,1.0,972.6,3,C57,2.1,1127.8,1,1009.4,2
45,45,NaN,2,pilot_3,940.4,3.0,1110.4,1,C57,2.2,965.6,3,970.6,4
46,46,NaN,2,pilot_3,952.3,2.0,1031.1,2,C57,2.3,1072.9,2,1000.6,3
47,47,NaN,2,pilot_3,938.2,4.0,885.9,4,C57,2.4,833.7,4,1019.4,1
48,48,NaN,3,pilot_3,1000.0,NaN,910.9,4,C57,3.1,847.8,4,1009.4,2
49,49,NaN,3,pilot_3,1000.0,NaN,1087.7,1,C57,3.2,1024.4,2,989.4,4


### Tone data

In [6]:
tone_times_df = pd.read_excel("/Users/kaavya/Documents/GitHub/social_dominance_active_inference/data/pilot_3/reward_competition/reward_comp_tone_sleap_frames.xlsx")

ALL_ITIS = [80, 90, 80, 75, 100, 95, 90, 80,
             90, 100, 90, 120, 90, 85, 90, 95, 
             120, 80, 95, 80, 90, 80, 75, 100, 95,
               90, 80, 90, 100, 90, 120, 90, 85, 90, 95, 90, 
               80, 90, 100, 90, 90, 90, 90, 90, 90, 90, 90, 90,
                 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90 ]
FRAME_RATE = 30
ALL_ITIS = [num * FRAME_RATE for num in ALL_ITIS]
ALL_TRIAL_NUMBERS = 19
tone_times_df = tone_times_df[tone_times_df["Frame Start"].astype(str).str.isnumeric()]

tone_times_df["all_tone_frame"] = tone_times_df.apply(lambda row: get_running_total([row["Frame Start"]] + ALL_ITIS), axis=1)

#code added for elo ranking calculation
tone_times_df['tube_test_RD'] = np.nan
tone_times_df['urine_marking_RD'] = np.nan
tone_times_df['home_cage_observation_RD'] = np.nan
#tone_times_df['tube_test_RD'] = pd.Series([pd.NA] * len(tone_times_df), dtype="Int64")
#tone_times_df['urine_marking_RD'] = pd.Series([pd.NA] * len(tone_times_df), dtype="Int64")
#tone_times_df['home_cage_observation_RD'] = pd.Series([pd.NA] * len(tone_times_df), dtype="Int64")
tone_times_df.set_index('File Name', inplace=True)

#function to calculate elo ranking difference of pairs and then append it to tone_times_df by file name
def ranking_difference(filtered_ranking_df, tone_times_df, id1, id2, ranking_col, file_name, tone_rd_col):
    ranking1 = filtered_ranking_df.loc[filtered_ranking_df['subject_id'] == id1, ranking_col].values[0]
    ranking2 = filtered_ranking_df.loc[filtered_ranking_df['subject_id'] == id2, ranking_col].values[0]
    difference = abs(ranking1 - ranking2)
    tone_times_df.loc[file_name, tone_rd_col] = difference
    
#code to loop through all the pairs 
combinations = [
    (1.1, 1.2, 'tube_test_rank', '10-03-22_Test_1_1-1v1-2', 'tube_test_RD'),
    (1.1, 1.2, 'urine_marking_rank', '10-03-22_Test_1_1-1v1-2', 'urine_marking_RD'),
    (2.1, 1.4, 'tube_test_rank', '10-03-22_Test_2_1-3v1-4', 'tube_test_RD'),
    (2.1, 1.4, 'urine_marking_rank', '10-03-22_Test_2_1-3v1-4', 'urine_marking_RD'),
    (3.2, 2.2, 'tube_test_rank', '10-03-22_Test_3_2-1v2-2', 'tube_test_RD'),
    (3.2, 2.2, 'urine_marking_rank', '10-03-22_Test_3_2-1v2-2', 'urine_marking_RD'),
    (2.2, 2.4, 'tube_test_rank', '10-03-22_Test_13_2-2v2-4', 'tube_test_RD'),
    (2.2, 2.4, 'urine_marking_rank', '10-03-22_Test_13_2-2v2-4', 'urine_marking_RD'),
    (2.2, 2.4, 'home_cage_observation_rank', '10-03-22_Test_13_2-2v2-4', 'home_cage_observation_RD'),
    (2.1, 2.3, 'tube_test_rank', '10-03-22_Test_14_2-1v2-3', 'tube_test_RD'),
    (2.1, 2.3, 'urine_marking_rank', '10-03-22_Test_14_2-1v2-3', 'urine_marking_RD'),
    (2.1, 2.3, 'home_cage_observation_rank', '10-03-22_Test_14_2-1v2-3', 'home_cage_observation_RD'),
    (1.2, 1.4, 'tube_test_rank', '10-03-22_Test_15_1-2v1-4', 'tube_test_RD'),
    (1.2, 1.4, 'urine_marking_rank', '10-03-22_Test_15_1-2v1-4', 'urine_marking_RD'),
    (1.1, 1.3, 'tube_test_rank', '10-03-22_Test_16_1-1v1-3', 'tube_test_RD'),
    (1.1, 1.3, 'urine_marking_rank', '10-03-22_Test_16_1-1v1-3', 'urine_marking_RD'),
    (3.2, 3.4, 'tube_test_rank', '10-03-22_Test_19_3-2v3-4', 'tube_test_RD'),
    (3.2, 3.4, 'urine_marking_rank', '10-03-22_Test_19_3-2v3-4', 'urine_marking_RD'),
    (3.1, 3.3, 'tube_test_rank', '10-03-22_Test_20_3-1v3-3', 'tube_test_RD'),
    (3.1, 3.3, 'urine_marking_rank', '10-03-22_Test_20_3-1v3-3', 'urine_marking_RD'),
    (3.1, 3.4, 'tube_test_rank', '10-04-22_Test_7_3-1v3-4', 'tube_test_RD'),
    (3.1, 3.4, 'urine_marking_rank', '10-04-22_Test_7_3-1v3-4', 'urine_marking_RD'),
    (3.2, 3.3, 'tube_test_rank', '10-04-22_Test_8_3-2v3-3', 'tube_test_RD'),
    (3.2, 3.3, 'urine_marking_rank', '10-04-22_Test_8_3-2v3-3', 'urine_marking_RD'),
    (2.1, 2.4, 'tube_test_rank', '10-04-22_Test_9_2-1v2-4', 'tube_test_RD'),
    (2.1, 2.4, 'urine_marking_rank', '10-04-22_Test_9_2-1v2-4', 'urine_marking_RD'),
    (2.1, 2.4, 'home_cage_observation_rank', '10-04-22_Test_9_2-1v2-4', 'home_cage_observation_RD'),
    (2.2, 2.3, 'tube_test_rank', '10-04-22_Test_10_2-2v2-3', 'tube_test_RD'),
    (2.2, 2.3, 'urine_marking_rank', '10-04-22_Test_10_2-2v2-3', 'urine_marking_RD'),
    (2.2, 2.3, 'home_cage_observation_rank', '10-04-22_Test_10_2-2v2-3', 'home_cage_observation_RD'),
    (1.1, 1.4, 'tube_test_rank', '10-04-22_Test_11_1-1v1-4', 'tube_test_RD'),
    (1.1, 1.4, 'urine_marking_rank', '10-04-22_Test_11_1-1v1-4', 'urine_marking_RD'),
    (1.2, 1.3, 'tube_test_rank', '10-04-22_Test_12_1-2v1-3', 'tube_test_RD'),
    (1.2, 1.3, 'urine_marking_rank', '10-04-22_Test_12_1-2v1-3', 'urine_marking_RD'),
    (3.1, 3.3, 'tube_test_rank', '10-04-22_Test_17_3-1v3-3', 'tube_test_RD'),
    (3.1, 3.3, 'urine_marking_rank', '10-04-22_Test_17_3-1v3-3', 'urine_marking_RD'),
    (1.1, 1.3, 'tube_test_rank', '10-04-22_Test_21_1-1v1-3', 'tube_test_RD'),
    (1.1, 1.3, 'urine_marking_rank', '10-04-22_Test_21_1-1v1-3', 'urine_marking_RD'),
    (4.1, 4.2, 'tube_test_rank', '10-03-22_Test_7_4-1v4-2', 'tube_test_RD'),
    (4.1, 4.2, 'urine_marking_rank', '10-03-22_Test_7_4-1v4-2', 'urine_marking_RD'),
    (4.1, 4.2, 'home_cage_observation_rank', '10-03-22_Test_7_4-1v4-2', 'home_cage_observation_RD'),
    (4.3, 4.4, 'tube_test_rank', '10-03-22_Test_8_4-3v4-4', 'tube_test_RD'),
    (4.3, 4.4, 'urine_marking_rank', '10-03-22_Test_8_4-3v4-4', 'urine_marking_RD'),
    (4.3, 4.4, 'home_cage_observation_rank', '10-03-22_Test_8_4-3v4-4', 'home_cage_observation_RD'),
    (6.1, 6.2, 'tube_test_rank', '10-03-22_Test_11_6-1v6-2', 'tube_test_RD'),
    (6.1, 6.2, 'urine_marking_rank', '10-03-22_Test_11_6-1v6-2', 'urine_marking_RD'),
    (6.3, 6.4, 'tube_test_rank', '10-03-22_Test_12_6-3v6-4', 'tube_test_RD'),
    (6.3, 6.4, 'urine_marking_rank', '10-03-22_Test_12_6-3v6-4', 'urine_marking_RD'),
    (5.2, 5.4, 'tube_test_rank', '10-03-22_Test_23_5-2v5-4', 'tube_test_RD'),
    (5.2, 5.4, 'urine_marking_rank', '10-03-22_Test_23_5-2v5-4', 'urine_marking_RD'),
    (5.2, 5.4, 'home_cage_observation_rank', '10-03-22_Test_23_5-2v5-4', 'home_cage_observation_RD'),
    (5.1, 5.3, 'tube_test_rank', '10-03-22_Test_24_5-1v5-3', 'tube_test_RD'),
    (5.1, 5.3, 'urine_marking_rank', '10-03-22_Test_24_5-1v5-3', 'urine_marking_RD'),
    (5.1, 5.3, 'home_cage_observation_rank', '10-03-22_Test_24_5-1v5-3', 'home_cage_observation_RD'),
    (4.1, 4.4, 'tube_test_rank', '10-03-22_Test_32_4-1v4-4', 'tube_test_RD'),
    (4.1, 4.4, 'urine_marking_rank', '10-03-22_Test_32_4-1v4-4', 'urine_marking_RD'),
    (4.1, 4.4, 'home_cage_observation_rank', '10-03-22_Test_32_4-1v4-4', 'home_cage_observation_RD'),
    (6.2, 6.3, 'tube_test_rank', '10-03-22_Test_35_6-2v6-3', 'tube_test_RD'),
    (6.2, 6.3, 'urine_marking_rank', '10-03-22_Test_35_6-2v6-3', 'urine_marking_RD'),
    (6.2, 6.3, 'tube_test_rank', '10-04-22_Test_2_6-2v6-3', 'tube_test_RD'),
    (6.2, 6.3, 'urine_marking_rank', '10-04-22_Test_2_6-2v6-3', 'urine_marking_RD'),
    (5.1, 5.4, 'tube_test_rank', '10-04-22_Test_3_5-1v5-4', 'tube_test_RD'),
    (5.1, 5.4, 'urine_marking_rank', '10-04-22_Test_3_5-1v5-4', 'urine_marking_RD'),
    (5.1, 5.4, 'home_cage_observation_rank', '10-04-22_Test_3_5-1v5-4', 'home_cage_observation_RD'),
    (5.1, 5.3, 'tube_test_rank', '10-04-22_Test_13_5-1v5-3', 'tube_test_RD'),
    (5.1, 5.3, 'urine_marking_rank', '10-04-22_Test_13_5-1v5-3', 'urine_marking_RD'),
    (5.1, 5.3, 'home_cage_observation_rank', '10-04-22_Test_13_5-1v5-3', 'home_cage_observation_RD'),
    (5.2, 5.4, 'tube_test_rank', '10-04-22_Test_14_5-2v5-4', 'tube_test_RD'),
    (5.2, 5.4, 'urine_marking_rank', '10-04-22_Test_14_5-2v5-4', 'urine_marking_RD'),
    (5.2, 5.4, 'home_cage_observation_rank', '10-04-22_Test_14_5-2v5-4', 'home_cage_observation_RD'),
    (6.1, 6.3, 'tube_test_rank', '10-04-22_Test_15_6-1v6-3', 'tube_test_RD'),
    (6.1, 6.3, 'urine_marking_rank', '10-04-22_Test_15_6-1v6-3', 'urine_marking_RD'),
    (6.2, 6.4, 'tube_test_rank', '10-04-22_Test_16_6-2v6-4', 'tube_test_RD'),
    (6.2, 6.4, 'urine_marking_rank', '10-04-22_Test_16_6-2v6-4', 'urine_marking_RD'),
    (4.1, 4.3, 'tube_test_rank', '10-04-22_Test_19_4-1v4-3', 'tube_test_RD'),
    (4.1, 4.3, 'urine_marking_rank', '10-04-22_Test_19_4-1v4-3', 'urine_marking_RD'),
    (4.1, 4.3, 'home_cage_observation_rank', '10-04-22_Test_19_4-1v4-3', 'home_cage_observation_RD'),
    (4.2, 4.4, 'tube_test_rank', '10-04-22_Test_20_4-2v4-4', 'tube_test_RD'),
    (4.2, 4.4, 'urine_marking_rank', '10-04-22_Test_20_4-2v4-4', 'urine_marking_RD'),
    (4.2, 4.4, 'home_cage_observation_rank', '10-04-22_Test_20_4-2v4-4', 'home_cage_observation_RD'),
        
]



for combination in combinations:
    id1, id2, ranking_col, file_name, tone_rd_col = combination
    ranking_difference(filtered_ranking_df, tone_times_df, id1, id2, ranking_col, file_name, tone_rd_col)

tone_times_df

#APPEND TO THIS DF

,Frame Start,FPS,Strain,all_tone_frame,tube_test_RD,urine_marking_RD,home_cage_observation_RD
File Name,,,,,,,
10-03-22_Test_1_1-1v1-2,2135,30,C57,"[2135, 4535, 7235, 9635, 11885, 14885, 17735, ...",1.0,1.0,NaN
10-03-22_Test_2_1-3v1-4,2123,29,C57,"[2123, 4523, 7223, 9623, 11873, 14873, 17723, ...",2.0,1.0,NaN
10-03-22_Test_3_2-1v2-2,2125,29,C57,"[2125, 4525, 7225, 9625, 11875, 14875, 17725, ...",1.0,0.0,NaN
10-03-22_Test_13_2-2v2-4,2138,30,C57,"[2138, 4538, 7238, 9638, 11888, 14888, 17738, ...",1.0,3.0,1.0
10-03-22_Test_14_2-1v2-3,2127,29,C57,"[2127, 4527, 7227, 9627, 11877, 14877, 17727, ...",1.0,1.0,1.0
10-03-22_Test_15_1-2v1-4,2126,29,C57,"[2126, 4526, 7226, 9626, 11876, 14876, 17726, ...",1.0,2.0,NaN
10-03-22_Test_16_1-1v1-3,2139,30,C57,"[2139, 4539, 7239, 9639, 11889, 14889, 17739, ...",3.0,2.0,NaN
10-03-22_Test_19_3-2v3-4,2090,30,C57,"[2090, 4490, 7190, 9590, 11840, 14840, 17690, ...",1.0,1.0,NaN
10-03-22_Test_20_3-1v3-3,2100,30,C57,"[2100, 4500, 7200, 9600, 11850, 14850, 17700, ...",1.0,1.0,NaN


### Sleap tracking

In [8]:
tone_times_df = tone_times_df.reset_index().rename(columns={"index":"File name"})		
tone_times_df

,File name,File Name,Frame Start,FPS,Strain,all_tone_frame,tube_test_RD,urine_marking_RD,home_cage_observation_RD
0,0,10-03-22_Test_1_1-1v1-2,2135,30,C57,"[2135, 4535, 7235, 9635, 11885, 14885, 17735, ...",1.0,1.0,NaN
1,1,10-03-22_Test_2_1-3v1-4,2123,29,C57,"[2123, 4523, 7223, 9623, 11873, 14873, 17723, ...",2.0,1.0,NaN
2,2,10-03-22_Test_3_2-1v2-2,2125,29,C57,"[2125, 4525, 7225, 9625, 11875, 14875, 17725, ...",1.0,0.0,NaN
3,3,10-03-22_Test_13_2-2v2-4,2138,30,C57,"[2138, 4538, 7238, 9638, 11888, 14888, 17738, ...",1.0,3.0,1.0
4,4,10-03-22_Test_14_2-1v2-3,2127,29,C57,"[2127, 4527, 7227, 9627, 11877, 14877, 17727, ...",1.0,1.0,1.0
5,5,10-03-22_Test_15_1-2v1-4,2126,29,C57,"[2126, 4526, 7226, 9626, 11876, 14876, 17726, ...",1.0,2.0,NaN
6,6,10-03-22_Test_16_1-1v1-3,2139,30,C57,"[2139, 4539, 7239, 9639, 11889, 14889, 17739, ...",3.0,2.0,NaN
7,7,10-03-22_Test_19_3-2v3-4,2090,30,C57,"[2090, 4490, 7190, 9590, 11840, 14840, 17690, ...",1.0,1.0,NaN
8,8,10-03-22_Test_20_3-1v3-3,2100,30,C57,"[2100, 4500, 7200, 9600, 11850, 14850, 17700, ...",1.0,1.0,NaN
9,9,10-04-22_Test_7_3-1v3-4,1838,30,C57,"[1838, 4238, 6938, 9338, 11588, 14588, 17438, ...",3.0,1.0,NaN


In [9]:
path = "/Users/kaavya/Documents/GitHub/social_dominance_active_inference/data/pilot_3/reward_competition/sleap_pose_tracking"
ending = "*.h5"
search_path = os.path.join(path, ending)
info ={}
total_frames = 0
for filename in glob.glob(search_path):
    file = os.path.basename(filename)
    file = file[:file.index('.')]
    info[file] = sleap_vid(filename, track_no=2)
    total_frames = info[file].locations.shape[0] + total_frames
    result_series = tone_times_df.loc[tone_times_df['File Name'] == file,'Strain']
    tones = tone_times_df.loc[tone_times_df['File Name'] == file,'all_tone_frame']
    tones = tones[0:ALL_TRIAL_NUMBERS]
    info[file].strain = result_series.iloc[0] if not result_series.empty else None
    info[file].tones = tones.iloc[0] if not tones.empty else None
    info[file].smooth_locations()
    print(info[file].tones, file)

[2057, 4457, 7157, 9557, 11807, 14807, 17657, 20357, 22757, 25457, 28457, 31157, 34757, 37457, 40007, 42707, 45557, 49157, 51557, 54407, 56807, 59507, 61907, 64157, 67157, 70007, 72707, 75107, 77807, 80807, 83507, 87107, 89807, 92357, 95057, 97907, 100607, 103007, 105707, 108707, 111407, 114107, 116807, 119507, 122207, 124907, 127607, 130307, 133007, 135707, 138407, 141107, 143807, 146507, 149207, 151907, 154607, 157307, 160007, 162707] 10-03-22_Test_23_5-2v5-4
[2017, 4417, 7117, 9517, 11767, 14767, 17617, 20317, 22717, 25417, 28417, 31117, 34717, 37417, 39967, 42667, 45517, 49117, 51517, 54367, 56767, 59467, 61867, 64117, 67117, 69967, 72667, 75067, 77767, 80767, 83467, 87067, 89767, 92317, 95017, 97867, 100567, 102967, 105667, 108667, 111367, 114067, 116767, 119467, 122167, 124867, 127567, 130267, 132967, 135667, 138367, 141067, 143767, 146467, 149167, 151867, 154567, 157267, 159967, 162667] 10-04-22_Test_12_1-2v1-3
[2125, 4525, 7225, 9625, 11875, 14875, 17725, 20425, 22825, 25525, 2

[2006, 4406, 7106, 9506, 11756, 14756, 17606, 20306, 22706, 25406, 28406, 31106, 34706, 37406, 39956, 42656, 45506, 49106, 51506, 54356, 56756, 59456, 61856, 64106, 67106, 69956, 72656, 75056, 77756, 80756, 83456, 87056, 89756, 92306, 95006, 97856, 100556, 102956, 105656, 108656, 111356, 114056, 116756, 119456, 122156, 124856, 127556, 130256, 132956, 135656, 138356, 141056, 143756, 146456, 149156, 151856, 154556, 157256, 159956, 162656] 10-04-22_Test_14_5-2v5-4
[2015, 4415, 7115, 9515, 11765, 14765, 17615, 20315, 22715, 25415, 28415, 31115, 34715, 37415, 39965, 42665, 45515, 49115, 51515, 54365, 56765, 59465, 61865, 64115, 67115, 69965, 72665, 75065, 77765, 80765, 83465, 87065, 89765, 92315, 95015, 97865, 100565, 102965, 105665, 108665, 111365, 114065, 116765, 119465, 122165, 124865, 127565, 130265, 132965, 135665, 138365, 141065, 143765, 146465, 149165, 151865, 154565, 157265, 159965, 162665] 10-03-22_Test_32_4-1v4-4
[2010, 4410, 7110, 9510, 11760, 14760, 17610, 20310, 22710, 25410, 2

## Feature Extraction

### Feature List


1. distance between thoraxes
2. velocity of mouse 1 + velocity of mouse 2 
3. | velocity of mouse 1 - velocity of mouse 2 |
4. sum of the angles of orientation between mouse 1 to mouse 2 and mouse 2 to mouse 1
5. | difference of the angles of orientation between mouse 1 to mouse 2 and mouse 2 to mouse 1 | 
6. sum of angles of orientation of mice to port 
7. | difference of angles of orientation of mice to port | 
8. sum of distances to port of mice
9. | differences of distances to port of mice |

### Feature calculations

In [10]:
is_first = True
recording_labels = []
frame_indice_labels = []
strain_labels = []
tube_test_elo_labels = []
urine_marking_elo_labels = []
home_cage_observation_elo_labels = []
#trial length = 30 fps * 10 second tone
trial_length = 10*30

file_row = tone_times_df[tone_times_df['File Name'] == file]
for file, recording in info.items():
    #creating arrays for recording name, strain, frame indice, and trial indice (0 for iti, 0-10 for tone)
    recording_labels += [file] * recording.locations.shape[0]
    strain_labels += [recording.strain] * recording.locations.shape[0]
    #elo ranking labels
    # df -> file name -> goes through labels (tube_test_elo_labels, urine_marking_elo_labels, home_cage_observation_elo_labels)
    #get output rank difference as it iterates through df
    file_row = tone_times_df[tone_times_df['File Name'] == file]
    if not file_row.empty:
        tube_test_elo_diff = file_row['tube_test_RD'].values[0]
        urine_marking_elo_diff = file_row['urine_marking_RD'].values[0]
        home_cage_elo_diff = file_row['home_cage_observation_RD'].values[0]
    else:
        tube_test_elo_diff = 0
        urine_marking_elo_diff = 0
        home_cage_elo_diff = 0
        
    tube_test_elo_labels += [tube_test_elo_diff] * recording.locations.shape[0]
    urine_marking_elo_labels += [urine_marking_elo_diff] * recording.locations.shape[0]
    home_cage_observation_elo_labels += [home_cage_elo_diff] * recording.locations.shape[0]
    
    tube_test_elo_array = np.array(tube_test_elo_labels)
    urine_marking_elo_array = np.array(urine_marking_elo_labels)
    home_cage_observation_elo_array = np.array(home_cage_observation_elo_labels)
 

    
    frame_indice_labels = np.concatenate([frame_indice_labels,np.arange(0, recording.locations.shape[0])], axis = 0)
    temp_trial_indices = np.zeros(recording.locations.shape[0])
    #loading in normalization_factor since not all the videos are the same size / resolution
    distance_normalization_factor = box_setup[file]['distance_normalization_factor']
    #loading in reward_point (x,y)
    reward_port = box_setup[file]['reward_port']
    #creating the tone snippets from 0-10 for the frames during the tone
    for trial in recording.tones:
        try:
            temp_trial_indices[trial:trial+trial_length] = np.linspace(0,10,300)
        except ValueError:
            pass
    if is_first:
        trial_indices = temp_trial_indices
    else: 
        trial_indices = np.concatenate([trial_indices, temp_trial_indices])
    # features 1 - 2 distance between thoraxes and noses
    f1 = recording.distances_between_mice('thorax', distance_normalization_factor) # distances between mice works 
    #features 3-4: velocities of each mouse
    velocities = recording.node_velocity('thorax', normalization_factor = distance_normalization_factor)
    f2 = velocities[0] + velocities[1]# this one is from sleap so assuming this works
    f3 = np.abs(velocities[0] - velocities[1])
    #feature 5: angle of orientation between mice, 2pi is facing each other
    # 0 radians is not facing each other (or parallel)
    orientations = recording.orientation()
    f4 = orientations[0] + orientations [1]# this works
    f5 = np.abs(orientations[0] - orientations [1])
    #feature 6-7: angle from nose to forehead to reward port
    angle_to_port = recording.point_angles('nose', 'thorax', reward_port)
    distance_to_port = recording.distances_to_point('nose', reward_port, distance_normalization_factor)
    f6 = angle_to_port[0] + angle_to_port[1]# this works with thorax
    f7 = np.abs(angle_to_port[0] - angle_to_port[1])
    #feature 8-9: distance to reward point for each mosue
    f8 = distance_to_port[0] + distance_to_port[1] # this works
    f9 = np.abs(distance_to_port[0] - distance_to_port[1])
    if is_first:
        features = np.stack([f1, f2, f3, f4, f5, f6, f7, f8, f9])
    else:
        temp_features =  np.stack([f1, f2, f3, f4, f5, f6, f7, f8, f9])
        features = np.concatenate([features,temp_features], axis = 1)
    is_first = False
recording_labels = np.array(recording_labels)
strain_labels = np.array(strain_labels)
zscored_features = []
for i in range(features.shape[0]):
    mean = np.mean(features[i])
    std_dev = np.std(features[i])
    normalized = (features[i]-mean)/std_dev
    zscored_features.append(normalized)
features = np.concatenate([features, np.stack(zscored_features)], axis = 0)
feature_names = ['thorax distances', 'mouse velocity sum',
                'mouse velocity diff', 'orientation b/w mice sum',
                'orientation b/w mice diff', 'orientation to port sum',
                'orientation to port diff', 'distance to port sum',
                'distance to port diff']
for name in range(len(feature_names)):
    zscore_name = feature_names[name]+'_zscore'
    feature_names.append(zscore_name)
    
new_columns = ['Strain', 'Recording', 'frame indice', 'trial_indice','tube_test_elo', 'urine_marking_elo', 'home_cage_observation_elo']
#label_columns = ['tube_test_elo', 'urine_marking_elo', 'home_cage_observation_elo']
column_names = feature_names + new_columns 
#df['tube_test_elo'] = tube_test_elo_array
#df['urine_marking_elo'] = urine_marking_elo_array
#df['home_cage_observation_elo'] = home_cage_observation_elo_array
data = np.column_stack([features.T, 
                        strain_labels[:, None],
                        recording_labels[:, None],
                        frame_indice_labels[:, None],
                        trial_indices[:,None],
                        tube_test_elo_array[:, None],
                        urine_marking_elo_array[:, None],
                        home_cage_observation_elo_array[:, None]])
df = pd.DataFrame(data, columns = column_names)
#df['tube_test_elo'] = tube_test_elo_labels
#df['urine_marking_elo'] = urine_marking_elo_labels
#df['home_cage_observation_elo'] = home_cage_observation_elo_labels
reduced_frames = len(df) 
every_third_index = np.arange(0, reduced_frames, 2)
subsample_df = df.iloc[every_third_index]

In [11]:
reduced_frames = len(df) 
every_third_index = np.arange(0, reduced_frames, 2)
subsample_df = df.iloc[every_third_index]

In [12]:
subsample_df

,thorax distances,mouse velocity sum,mouse velocity diff,orientation b/w mice sum,orientation b/w mice diff,orientation to port sum,orientation to port diff,distance to port sum,distance to port diff,thorax distances_zscore,...,orientation to port diff_zscore,distance to port sum_zscore,distance to port diff_zscore,Strain,Recording,frame indice,trial_indice,tube_test_elo,urine_marking_elo,home_cage_observation_elo
0,71.0238096873941,4.5201478726659153e-13,8.075011250642511e-14,2.698060892789739,0.5900219842541166,1.823140590119341,1.6266988170102281,117.75594215313897,89.66951169413421,-0.7217941850247467,...,-0.007684828445971038,-0.42406493768425363,-0.3838177631495446,CD1,10-03-22_Test_23_5-2v5-4,0.0,0.0,2.0,1.0,1.0
2,71.02380968739408,3.213506087500488e-13,6.701074136036494e-14,2.6980608927897443,0.590021984254119,1.823140590119347,1.626698817010228,117.75594215313912,89.66951169413393,-0.7217941850247469,...,-0.007684828445971182,-0.4240649376842528,-0.38381776314954646,CD1,10-03-22_Test_23_5-2v5-4,2.0,0.0,2.0,1.0,1.0
4,71.02380968739403,2.143398346822626e-13,5.250784112023997e-14,2.698060892789745,0.5900219842541181,1.8231405901193496,1.6266988170102248,117.75594215313909,89.6695116941337,-0.7217941850247472,...,-0.007684828445973194,-0.42406493768425296,-0.38381776314954796,CD1,10-03-22_Test_23_5-2v5-4,4.0,0.0,2.0,1.0,1.0
6,71.02380968739403,1.3321959927136133e-13,3.577438517837657e-14,2.698060892789745,0.590021984254115,1.8231405901193494,1.6266988170102223,117.75594215313907,89.66951169413349,-0.7217941850247472,...,-0.007684828445974774,-0.424064937684253,-0.38381776314954935,CD1,10-03-22_Test_23_5-2v5-4,6.0,0.0,2.0,1.0,1.0
8,71.02380968739396,8.239909857723636e-14,1.4582122956432874e-14,2.698060892789746,0.5900219842541155,1.8231405901193505,1.6266988170102212,117.7559421531391,89.66951169413329,-0.7217941850247478,...,-0.007684828445975492,-0.42406493768425285,-0.38381776314955063,CD1,10-03-22_Test_23_5-2v5-4,8.0,0.0,2.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832816,506.8155992088786,0.5447743221527092,0.5447743221525131,2.914471898833778,2.4983156877694004,0.3011416908833803,0.1496450547542939,528.1842572698299,498.7124562291649,2.8645630358006473,...,-0.9634229542666335,1.9431667499450889,2.2944825053907274,C57,10-04-22_Test_9_2-1v2-4,55886.0,0.0,3.0,1.0,3.0
1832818,506.69924843148135,0.5426052987026073,0.542605298702419,2.931725856931411,2.5166168496896364,0.3038050746037725,0.1469816710339079,528.1845607174122,498.7121527815824,2.8636055247582166,...,-0.9651463156163077,1.9431685001428842,2.2944805184997357,C57,10-04-22_Test_9_2-1v2-4,55888.0,0.0,3.0,1.0,3.0
1832820,506.4732504657886,0.523059002751743,0.5230590027515691,2.95134249560176,2.5358752311159716,0.3077930074026002,0.14299373823508998,528.0580188692127,498.83869462978197,2.8617456699935273,...,-0.9677267358531582,1.9424386434009489,2.2953090795849285,C57,10-04-22_Test_9_2-1v2-4,55890.0,0.0,3.0,1.0,3.0
1832822,506.0806623897265,0.4866817108349468,0.4866817108347947,2.974608813581792,2.5569620007546785,0.3130408088135064,0.137745936824194,527.7987659586622,499.0979475403322,2.8585148585940257,...,-0.9711223630221166,1.9409433476760654,2.2970065960663235,C57,10-04-22_Test_9_2-1v2-4,55892.0,0.0,3.0,1.0,3.0


In [13]:
subsample_df['distance to port diff_zscore'] = np.log1p(subsample_df['distance to port diff_zscore'].astype(float)+2)
subsample_df['distance to port sum_zscore'] = np.log1p(subsample_df['distance to port sum_zscore'].astype(float)+2)
subsample_df['orientation to port diff_zscore'] = np.log1p(subsample_df['orientation to port diff_zscore'].astype(float)+2)
subsample_df['oreintation to port sum_zscore'] = np.log1p(subsample_df['orientation to port sum_zscore'].astype(float)+2)

In [14]:
subsample_df

,thorax distances,mouse velocity sum,mouse velocity diff,orientation b/w mice sum,orientation b/w mice diff,orientation to port sum,orientation to port diff,distance to port sum,distance to port diff,thorax distances_zscore,...,distance to port sum_zscore,distance to port diff_zscore,Strain,Recording,frame indice,trial_indice,tube_test_elo,urine_marking_elo,home_cage_observation_elo,oreintation to port sum_zscore
0,71.0238096873941,4.5201478726659153e-13,8.075011250642511e-14,2.698060892789739,0.5900219842541166,1.823140590119341,1.6266988170102281,117.75594215313897,89.66951169413421,-0.7217941850247467,...,0.946213,0.961716,CD1,10-03-22_Test_23_5-2v5-4,0.0,0.0,2.0,1.0,1.0,1.033423
2,71.02380968739408,3.213506087500488e-13,6.701074136036494e-14,2.6980608927897443,0.590021984254119,1.823140590119347,1.626698817010228,117.75594215313912,89.66951169413393,-0.7217941850247469,...,0.946213,0.961716,CD1,10-03-22_Test_23_5-2v5-4,2.0,0.0,2.0,1.0,1.0,1.033423
4,71.02380968739403,2.143398346822626e-13,5.250784112023997e-14,2.698060892789745,0.5900219842541181,1.8231405901193496,1.6266988170102248,117.75594215313909,89.6695116941337,-0.7217941850247472,...,0.946213,0.961716,CD1,10-03-22_Test_23_5-2v5-4,4.0,0.0,2.0,1.0,1.0,1.033423
6,71.02380968739403,1.3321959927136133e-13,3.577438517837657e-14,2.698060892789745,0.590021984254115,1.8231405901193494,1.6266988170102223,117.75594215313907,89.66951169413349,-0.7217941850247472,...,0.946213,0.961716,CD1,10-03-22_Test_23_5-2v5-4,6.0,0.0,2.0,1.0,1.0,1.033423
8,71.02380968739396,8.239909857723636e-14,1.4582122956432874e-14,2.698060892789746,0.5900219842541155,1.8231405901193505,1.6266988170102212,117.7559421531391,89.66951169413329,-0.7217941850247478,...,0.946213,0.961716,CD1,10-03-22_Test_23_5-2v5-4,8.0,0.0,2.0,1.0,1.0,1.033423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832816,506.8155992088786,0.5447743221527092,0.5447743221525131,2.914471898833778,2.4983156877694004,0.3011416908833803,0.1496450547542939,528.1842572698299,498.7124562291649,2.8645630358006473,...,1.598006,1.666665,C57,10-04-22_Test_9_2-1v2-4,55886.0,0.0,3.0,1.0,3.0,0.738853
1832818,506.69924843148135,0.5426052987026073,0.542605298702419,2.931725856931411,2.5166168496896364,0.3038050746037725,0.1469816710339079,528.1845607174122,498.7121527815824,2.8636055247582166,...,1.598007,1.666665,C57,10-04-22_Test_9_2-1v2-4,55888.0,0.0,3.0,1.0,3.0,0.739453
1832820,506.4732504657886,0.523059002751743,0.5230590027515691,2.95134249560176,2.5358752311159716,0.3077930074026002,0.14299373823508998,528.0580188692127,498.83869462978197,2.8617456699935273,...,1.597859,1.666821,C57,10-04-22_Test_9_2-1v2-4,55890.0,0.0,3.0,1.0,3.0,0.740349
1832822,506.0806623897265,0.4866817108349468,0.4866817108347947,2.974608813581792,2.5569620007546785,0.3130408088135064,0.137745936824194,527.7987659586622,499.0979475403322,2.8585148585940257,...,1.597556,1.667142,C57,10-04-22_Test_9_2-1v2-4,55892.0,0.0,3.0,1.0,3.0,0.741528


# Feature Visualization


In [ ]:

#Load your video using cv2.VideoCapture()
#pick random video file
cap = cv2.VideoCapture(r'C:\Users\megha\Documents\GitHub\social_dominance_active_inference\data\videos\10-03-22_Test_23_5-2v5-4.fixed.mp4')  # Replace 'your_video.mp4' with your video file
#insert video file base name 
file = '10-03-22_Test_23_5-2v5-4'
sleap = info[file]

#and pick feature 
feature = 'distance to port sum'
if not cap.isOpened(): 
    print("Could not open the video file!")
else:
    print("Video read successful!")
    success, image = cap.read()
    recording_df = df[df['Recording'] == file]
    my_array = np.array(recording_df[feature]) # Example array for demonstration
    frame_list = np.arange(1, sleap.locations.shape[0]+1)
    random_frames = np.random.choice(frame_list, size=10, replace=False)
    fig, axs = plt.subplots(5, 2, figsize=(15, 15))
    for idx, frame_num in enumerate(random_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num - 1)  # Frame indexing starts from 0
        success, image = cap.read()
        if success:
            # Convert BGR image to RGB for display with matplotlib
            frame_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            # Plot on the respective subplot
            row = idx // 2  # Calculate subplot row
            col = idx % 2  # Calculate subplot column
            axs[row, col].imshow(frame_rgb)
            axs[row, col].scatter(
                #pick nodes that you want to visualize, last dimenison is mouse
                #0 = mouse 1 , 1 = mouse 2 
                sleap.locations[frame_num, sleap.node_dict['nose'], 0, :],
                sleap.locations[frame_num, sleap.node_dict['nose'], 1, :],
                s=5
            )
            axs[row, col].scatter(box_setup[file]['reward_port'][0],
                                  box_setup[file]['reward_port'][1],s=5)
            # axs[row, col].scatter(box_setup[file]['bottom_left'][0],
            #                       box_setup[file]['bottom_left'][1],s=5)
            # axs[row, col].scatter(box_setup[file]['bottom_right'][0],
            #                       box_setup[file]['bottom_right'][1],s=5)
            # axs[row, col].scatter(
            #     sleap.locations[frame_num, sleap.node_dict['thorax'], 0, :],
            #     sleap.locations[frame_num, sleap.node_dict['thorax'], 1, :],
            #     s=5
            # )
            # axs[row, col].scatter(
            #     sleap.locations[frame_num, sleap.node_dict['forehead'], 0, :],
            #     sleap.locations[frame_num, sleap.node_dict['forehead'], 1, :],
            #     s=5)
            axs[row, col].axis('off')  # Turn off axis labels and ticks
            axs[row, col].set_title(f"Feature: {my_array[frame_num]}")
    plt.tight_layout()  # Adjust subplot layout
    plt.show()
cap.release()


# UMAPS


#Load your video using cv2.VideoCapture()
#pick random video file
cap = cv2.VideoCapture(r'C:\Users\megha\Documents\GitHub\social_dominance_active_inference\data\videos\10-03-22_Test_23_5-2v5-4.fixed.mp4')  # Replace 'your_video.mp4' with your video file
#insert video file base name 
file = '10-03-22_Test_23_5-2v5-4'
sleap = info[file]

#and pick feature 
feature = ''
try:
    if not cap.isOpened(): 
        print("Could not open the video file!")
    else:
        print("Video read successful!")
        success, image = cap.read()
        recording_df = df[df['Recording'] == file]
        my_array = np.array(recording_df[feature]) # Example array for demonstration
        frame_list = np.arange(1, sleap.locations.shape[0]+1)
        random_frames = np.random.choice(frame_list, size=10, replace=False)
        fig, axs = plt.subplots(5, 2, figsize=(15, 15))
        for idx, frame_num in enumerate(random_frames):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num - 1)  # Frame indexing starts from 0
            success, image = cap.read()
            
            if success:
                # Convert BGR image to RGB for display with matplotlib
                frame_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                # Plot on the respective subplot
                row = idx // 2  # Calculate subplot row
                col = idx % 2  # Calculate subplot column
                axs[row, col].imshow(frame_rgb)
                axs[row, col].scatter(
                    #pick nodes that you want to visualize, last dimenison is mouse
                    #0 = mouse 1 , 1 = mouse 2 
                    sleap.locations[frame_num, sleap.node_dict['nose'], 0, :],
                    sleap.locations[frame_num, sleap.node_dict['nose'], 1, :],
                    s=5
                )
                axs[row, col].scatter(box_setup[file]['bottom_left'][0],
                                    box_setup[file]['bottom_left'][1],s=5)
                axs[row, col].scatter(box_setup[file]['bottom_right'][0],
                                    box_setup[file]['bottom_right'][1],s=5)
                axs[row, col].scatter(
                    sleap.locations[frame_num, sleap.node_dict['thorax'], 0, :],
                    sleap.locations[frame_num, sleap.node_dict['thorax'], 1, :],
                    s=5
                )
                axs[row, col].scatter(
                    sleap.locations[frame_num, sleap.node_dict['forehead'], 0, 1],
                    sleap.locations[frame_num, sleap.node_dict['forehead'], 1, 1],
                    s=5)
                axs[row, col].axis('off')  # Turn off axis labels and ticks
                axs[row, col].set_title(f"Feature: {my_array[frame_num]}")
        plt.tight_layout()  # Adjust subplot layout
        plt.show()
    cap.release()
except: 
    print('check feature and file name')


### Color dictionary for labels

In [15]:
def make_color_dict(column_name, df, cmap = 'gist_ncar'):
    unique_labels = df[column_name].unique()
    color_map = plt.cm.get_cmap(cmap, len(unique_labels))
    # Create a dictionary to map labels to colors
    label_color_dict = {label: color_map(i) for i, label in enumerate(unique_labels)}
    # Assign colors to each label based on the color map
    #colors = [label_color_dict[label] for label in recording_labels]
    #colors = np.array(colors)
    return label_color_dict

def show_palette(color_dict):
    colors = list(color_dict.values())
    palette = sns.color_palette(colors)
    sub_pallete_no = int(len(palette)//5)
    divided_palettes = [palette[i:i+5] for i in range(0, len(palette), 5)]  # Create 6 lists of 5 elements
    divided_palettes.append(palette[sub_pallete_no:])
    # Plotting color swatches for each label
    for color_palette in divided_palettes:
        sns.palplot(color_palette)  # Plotting color swatches for each sub-palette
        
        # Adding labels above the color swatches
        plt.xticks([])  # Hide x-axis ticks
        for color, label in zip(color_palette, list(color_dict.keys())):
            plt.text(color_palette.index(color), -0.5, label, ha='center', fontsize=8, rotation=45)

        plt.show()
recording_colors = make_color_dict('Recording', subsample_df)
strain_colors = {'C57': 'r', 'CD1':'b'}

## UMAP calls

In [15]:
subsample_df

,thorax distances,mouse velocity sum,mouse velocity diff,orientation b/w mice sum,orientation b/w mice diff,orientation to port sum,orientation to port diff,distance to port sum,distance to port diff,thorax distances_zscore,...,distance to port sum_zscore,distance to port diff_zscore,Strain,Recording,frame indice,trial_indice,tube_test_elo,urine_marking_elo,home_cage_observation_elo,oreintation to port sum_zscore
0,71.0238096873941,4.5201478726659153e-13,8.075011250642511e-14,2.698060892789739,0.5900219842541166,1.823140590119341,1.6266988170102281,117.75594215313897,89.66951169413421,-0.7217941850247467,...,0.946213,0.961716,CD1,10-03-22_Test_23_5-2v5-4,0.0,0.0,2.0,1.0,1.0,1.033423
3,71.02380968739408,2.647975092461169e-13,5.991400005770124e-14,2.6980608927897456,0.5900219842541179,1.8231405901193494,1.626698817010226,117.75594215313912,89.66951169413373,-0.7217941850247469,...,0.946213,0.961716,CD1,10-03-22_Test_23_5-2v5-4,3.0,0.0,2.0,1.0,1.0,1.033423
6,71.02380968739403,1.3321959927136133e-13,3.577438517837657e-14,2.698060892789745,0.590021984254115,1.8231405901193494,1.6266988170102223,117.75594215313907,89.66951169413349,-0.7217941850247472,...,0.946213,0.961716,CD1,10-03-22_Test_23_5-2v5-4,6.0,0.0,2.0,1.0,1.0,1.033423
9,71.02380968739394,6.867772540411011e-14,3.690354810261768e-15,2.6980608927897456,0.5900219842541139,1.8231405901193505,1.6266988170102203,117.75594215313905,89.66951169413323,-0.721794185024748,...,0.946213,0.961716,CD1,10-03-22_Test_23_5-2v5-4,9.0,0.0,2.0,1.0,1.0,1.033423
12,71.02380968739392,1.032719863128999e-13,1.8634005320461304e-14,2.6980608927897585,0.5900219842541248,1.8231405901193534,1.6266988170102432,117.75594215313903,89.66951169413447,-0.7217941850247481,...,0.946213,0.961716,CD1,10-03-22_Test_23_5-2v5-4,12.0,0.0,2.0,1.0,1.0,1.033423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832811,506.8905397528991,0.4971469037223386,0.4971469037221644,2.8664409020596615,2.444293375585522,0.3014569966200463,0.14932974901763985,527.80082144979,499.0958920492058,2.865179760501853,...,1.597559,1.667139,C57,10-04-22_Test_9_2-1v2-4,55881.0,0.0,3.0,1.0,3.0,0.738924
1832814,506.8795783566121,0.5295461593126641,0.5295461593124676,2.898279158789676,2.4800842516307684,0.29977939192825565,0.1510073537094141,528.064565764921,498.8321477340738,2.865089553472557,...,1.597867,1.666813,C57,10-04-22_Test_9_2-1v2-4,55884.0,0.0,3.0,1.0,3.0,0.738547
1832817,506.7675569480669,0.545874132859445,0.5458741328592518,2.922884540128932,2.5074017186287505,0.3023078282326387,0.1484789174050386,528.1998379039406,498.69687559505394,2.864167671057415,...,1.598024,1.666646,C57,10-04-22_Test_9_2-1v2-4,55887.0,0.0,3.0,1.0,3.0,0.739116
1832820,506.4732504657886,0.523059002751743,0.5230590027515691,2.95134249560176,2.5358752311159716,0.3077930074026002,0.14299373823508998,528.0580188692127,498.83869462978197,2.8617456699935273,...,1.597859,1.666821,C57,10-04-22_Test_9_2-1v2-4,55890.0,0.0,3.0,1.0,3.0,0.740349


In [16]:
zscore_df = subsample_df.iloc[:,9:18]
zscore_df
elo_df = subsample_df.iloc[:,22:24]
zscore_df

,thorax distances_zscore,mouse velocity sum_zscore,mouse velocity diff_zscore,orientation b/w mice sum_zscore,orientation b/w mice diff_zscore,orientation to port sum_zscore,orientation to port diff_zscore,distance to port sum_zscore,distance to port diff_zscore
0,-0.7217941850247467,-0.7936327099738386,-0.6478233911854566,-0.4268892232434471,-0.5357052093404989,-0.1893294033086247,1.096047,0.946213,0.961716
2,-0.7217941850247469,-0.7936327099738771,-0.6478233911854616,-0.426889223243445,-0.5357052093404971,-0.18932940330862186,1.096047,0.946213,0.961716
4,-0.7217941850247472,-0.7936327099739087,-0.6478233911854671,-0.42688922324344464,-0.5357052093404978,-0.18932940330862058,1.096047,0.946213,0.961716
6,-0.7217941850247472,-0.7936327099739325,-0.6478233911854732,-0.42688922324344464,-0.5357052093405,-0.1893294033086207,1.096047,0.946213,0.961716
8,-0.7217941850247478,-0.7936327099739475,-0.6478233911854809,-0.4268892232434443,-0.5357052093404997,-0.18932940330862016,1.096047,0.946213,0.961716
...,...,...,...,...,...,...,...,...,...
1832816,2.8645630358006473,-0.6331594692360035,-0.44783365357572613,-0.34311451136376075,0.8499815079521552,-0.9064662640825379,0.711270,1.598006,1.666665
1832818,2.8636055247582166,-0.633798394765568,-0.4486299143630183,-0.3364353438166202,0.863270697628345,-0.9052113284572643,0.710424,1.598007,1.666665
1832820,2.8617456699935273,-0.63955611374774,-0.4558054702574439,-0.32884156065975995,0.8772549618377115,-0.9033322905619253,0.709155,1.597859,1.666821
1832822,2.8585148585940257,-0.6502717104375617,-0.46915978005061487,-0.3198349527200431,0.8925668902610671,-0.9008596265989807,0.707483,1.597556,1.667142


### Enhanced Clusterable UMAP

In [ ]:
clusterable_embedding = umap.UMAP(
    n_neighbors= 18,
    min_dist=0,
    n_components=2,
    random_state=42,
).fit_transform(zscore_df)

In [ ]:
plt.scatter(clusterable_embedding[:, 0], clusterable_embedding[:, 1])

### Standard Embedding UMAP

In [ ]:

standard_embedding = umap.UMAP(random_state=42).fit_transform(zscore_df)


In [ ]:
plt.scatter(standard_embedding[:, 0], standard_embedding[:, 1])

In [57]:
# subsample_df['embedding_x'] = clusterable_embedding[:,0]
# subsample_df['embedding_y']= clusterable_embedding[:,1]
subsample_df['embedding_x_standard'] = standard_embedding[:,0]
subsample_df['embedding_y_standard'] = standard_embedding[:,1]

### Clustering Calls

In [ ]:
# hdbscan_labels = hdbscan.HDBSCAN(min_samples=1, min_cluster_size=7000, max_cluster_size=460000).fit_predict(clusterable_embedding)
# kmeans_labels = cluster.KMeans(n_clusters=12).fit_predict(clusterable_embedding)


In [58]:
# standard_hdbscan_labels = hdbscan.HDBSCAN(min_samples=1, min_cluster_size=7000, max_cluster_size=450000).fit_predict(standard_embedding)
standard_kmeans_labels = cluster.KMeans(n_clusters=10).fit_predict(standard_embedding)

In [59]:
kmeans_raw_label = cluster.KMeans(n_clusters=10).fit_predict(elo_df)

In [60]:
len(kmeans_raw_label)

610942

In [64]:
subsample_df['kmeans_label'] = standard_kmeans_labels
subsample_df['hdbscan_labels'] =  standard_hdbscan_labels
# subsample_df['enhanced_kmeans_labels'] = kmeans_labels
# subsample_df['enhanced_hdbscan_labels'] = hdbscan_labels

clustered= (hdbscan_labels >= 0)
clustered_df = subsample_df#[subsample_df['cluster_label']]>= 0]
clustered_dataset = len(clustered_df) 
random_subsample = np.random.choice(clustered_dataset, size=(clustered_dataset//10), replace = False)
subsample_cluster_df = clustered_df.iloc[random_subsample]
trial_df = clustered_df[clustered_df['trial_indice'].astype(float) > 0]
unclustered = subsample_df[subsample_df['hdbscan_labels']==-1]

### Pickle the results

In [ ]:
# with open('clustered_df.pkl', 'wb') as file:
#     pickle.dump(clustered_df, file)
with open('clustered_df.pkl', 'rb') as file:
    clustered_df = pickle.load(file)

In [ ]:
clustered_df['raw_kmeans_labels'] = kmeans_raw_label
trial_df = clustered_df[clustered_df['trial_indice'].astype(float) > 0]

## Cluster plots

### Standard Embedding

In [65]:
clustered_dataset = len(clustered_df) 
random_subsample = np.random.choice(clustered_dataset, size=(clustered_dataset//10), replace = False)
subsample_cluster_df = clustered_df.iloc[random_subsample]
plt.rcdefaults()

In [66]:
plt.figure(figsize=(5,5))
scatter = plt.scatter(subsample_cluster_df['embedding_x_standard'],
            subsample_cluster_df['embedding_y_standard'],
            c=subsample_cluster_df['raw_kmeans_labels'],#.map(cluster_colors),
            label=subsample_cluster_df['raw_kmeans_labels'],
            s=0.1,
            cmap='Spectral'
            )

#cluster_labels = []

plt.title('Behavioral Clusters')
plt.legend(*scatter.legend_elements(), bbox_to_anchor = (1,1))
#for cluster, color in cluster_colors.items():
#    cluster_labels.append(mpatches.Patch(facecolor=color,  label=cluster))

#plt.legend(handles = cluster_labels, bbox_to_anchor = (1,1))
plt.show()

KeyError: 'raw_kmeans_labels'

<Figure size 500x500 with 0 Axes>

In [ ]:
plt.figure(figsize=(5,5))
scatter = plt.scatter(subsample_cluster_df['embedding_x_standard'],
            subsample_cluster_df['embedding_y_standard'],
            c=subsample_cluster_df['kmeans_label'].map(cluster_colors),
            label=subsample_cluster_df['kmeans_label'],
            s=0.1)
cluster_labels = []
for cluster, color in cluster_colors.items():
    cluster_labels.append(mpatches.Patch(facecolor=color,  label=cluster))
plt.title('Behavioral Clusters')
plt.legend(handles = cluster_labels, bbox_to_anchor = (1,1))
plt.show()

### Enhanced Embedding

In [ ]:
plt.figure(figsize=(5,5))
scatter = plt.scatter(subsample_cluster_df['embedding_x'],
            subsample_cluster_df['embedding_y'],
            c=subsample_cluster_df['enhanced_kmeans_labels'],
            label=subsample_cluster_df['enhanced_kmeans_labels'],
            s=0.1,
            cmap='Spectral')
plt.title('Behavioral Clusters')
plt.legend(*scatter.legend_elements(), bbox_to_anchor = (1,1))
plt.show()

### Feature overlay plots

In [ ]:
from matplotlib.patches import Patch
fig, ax = plt.subplots(4, 3, figsize=(15, 20))
scatter1 = ax[0,0].scatter(trial_df['embedding_x_standard'],
            trial_df['embedding_y_standard'],
            c=trial_df['trial_indice'].astype(float),
            s=0.1,
            cmap='cividis')
ax[0,0].set_title('Trial: Time from Tone Onset', fontsize= 16)

scatter2 = ax[0,1].scatter(subsample_cluster_df['embedding_x_standard'],
            subsample_cluster_df['embedding_y_standard'],
            c=subsample_cluster_df['Recording'].map(recording_colors),
            label = subsample_cluster_df['Recording'],
            s=0.005)   
ax[0,1].set_title('Recordings', fontsize= 16)
legend2 = ax[0,1].legend(*scatter2.legend_elements(), bbox_to_anchor = (1,1))

scatter3 = ax[0,2].scatter(subsample_cluster_df['embedding_x_standard'],
            subsample_cluster_df['embedding_y_standard'],
            c=subsample_cluster_df['Strain'].map(strain_colors),
            label=subsample_cluster_df['Strain'],
            s=0.005)  
ax[0,2].set_title('Strains', fontsize= 16)
legend_handles3 = [Patch(color=color, label=label) for label, color in strain_colors.items()]
legend3 = ax[0,2].legend(handles = legend_handles3, bbox_to_anchor = (1,1))

scatter4 = ax[1,0].scatter(subsample_cluster_df['embedding_x_standard'],
            subsample_cluster_df['embedding_y_standard'],
            c=subsample_cluster_df['thorax distances'].astype(float),
            s=0.005, cmap = 'afmhot', vmax = 400)  
ax[1,0].set_title('Distance between mice', fontsize= 16)

scatter5 = ax[1,1].scatter(subsample_cluster_df['embedding_x_standard'],
            subsample_cluster_df['embedding_y_standard'],
            c=subsample_cluster_df['distance to port sum_zscore'].astype(float),
            s=0.005, cmap = 'afmhot', vmax = 6)  
ax[1,1].set_title('Sum of distance to port', fontsize= 16)

scatter6 = ax[1,2].scatter(subsample_cluster_df['embedding_x_standard'],
            subsample_cluster_df['embedding_y_standard'],
            c=subsample_cluster_df['distance to port diff_zscore'].astype(float),
            s=0.005, cmap = 'afmhot', vmax = 6)  
ax[1,2].set_title('Difference in distance to port', fontsize= 16)

scatter7 = ax[2,0].scatter(subsample_cluster_df['embedding_x_standard'],
            subsample_cluster_df['embedding_y_standard'],
            c=subsample_cluster_df['orientation to port sum'].astype(float),
            s=0.005, cmap = 'afmhot', vmax=7)  
ax[2,0].set_title('Sum of orientation to port', fontsize= 16)

scatter8 = ax[2,1].scatter(subsample_cluster_df['embedding_x_standard'],
            subsample_cluster_df['embedding_y_standard'],
            c=subsample_cluster_df['orientation to port diff'].astype(float),
            s=0.005, cmap = 'afmhot', vmax = 4)  
ax[2,1].set_title('Diff in orientation to port', fontsize= 16)

scatter9 = ax[2,2].scatter(subsample_cluster_df['embedding_x_standard'],
            subsample_cluster_df['embedding_y_standard'],
            c=subsample_cluster_df['orientation b/w mice sum'].astype(float),
            s=0.005, cmap = 'afmhot', vmax = 8)  
ax[2,2].set_title('Sum of orientation between mice', fontsize= 16)


scatter10 = ax[3,2].scatter(subsample_cluster_df['embedding_x_standard'],
            subsample_cluster_df['embedding_y_standard'],
            c=subsample_cluster_df['orientation b/w mice diff'].astype(float),
            s=0.005, cmap = 'afmhot', vmax = 4)  
ax[3,2].set_title('Diff in orientation between mice', fontsize= 16)

scatter11 = ax[3,0].scatter(subsample_cluster_df['embedding_x_standard'],
            subsample_cluster_df['embedding_y_standard'],
            c=subsample_cluster_df['mouse velocity sum'].astype(float),
            s=0.005, cmap = 'afmhot', vmax = 20)  
ax[3,0].set_title('Sum of velocities between mice', fontsize= 16)

scatter12 = ax[3,1].scatter(subsample_cluster_df['embedding_x_standard'],
            subsample_cluster_df['embedding_y_standard'],
            c=subsample_cluster_df['mouse velocity diff'].astype(float),
            s=0.005, cmap = 'afmhot', vmax = 12.5)  
ax[3,1].set_title('Diff in velocities between mice', fontsize= 16)


plt.colorbar(scatter1, ax=ax[0,0]) 
plt.colorbar(scatter4, ax=ax[1,0]) 
plt.colorbar(scatter5, ax=ax[1,1]) 
plt.colorbar(scatter6, ax=ax[1,2]) 
plt.colorbar(scatter7, ax=ax[2,0]) 
plt.colorbar(scatter8, ax=ax[2,1]) 
plt.colorbar(scatter9, ax=ax[2,2]) 
plt.colorbar(scatter10, ax=ax[3,2]) 
plt.colorbar(scatter11, ax=ax[3,0]) 
plt.colorbar(scatter12, ax=ax[3,1]) 

#legend_handles4 = [Patch(color=color, label=label) for label, color in strain_colors.items()]
#legend3 = ax[3].legend(handles = legend_handles3)
plt.savefig('./feature_clusters.png')
plt.show()


In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 20))
scatter1 = ax[0,0].scatter(subsample_cluster_df['embedding_x_standard'],
            subsample_cluster_df['embedding_y_standard'],
            c=subsample_cluster_df['tube_test_elo'].astype(float),
            s=0.005,
            cmap='cividis')
ax[0,0].set_title('Trial: Time from Tone Onset', fontsize= 16)

scatter2 = ax[0,1].scatter(subsample_cluster_df['embedding_x_standard'],
            subsample_cluster_df['embedding_y_standard'],
            c=subsample_cluster_df['urine_marking_elo' ].astype(float),
            s=0.005,
            cmap='cividis')
#legend_handles3 = []
                           
#legend3 = ax[0,2].legend(handles = legend_handles3, bbox_to_anchor = (1,1)),
            #s=0.005)   
ax[0,1].set_title('Recordings', fontsize= 16)
#legend2 = ax[0,1].legend(*scatter2.legend_elements(), bbox_to_anchor = (1,1))

#scatter3 = ax[0,2].scatter(subsample_cluster_df['embedding_x_standard'],
            #subsample_cluster_df['embedding_y_standard'],
            #c=subsample_cluster_df['home_cage_observation_elo' ].astype(float),
            #s=0.005,
            #cmap='cividis')  
#ax[0,2].set_title('Strains', fontsize= 16)


In [ ]:
from matplotlib.patches import Patch
fig, ax = plt.subplots(4, 3, figsize=(15, 20))
scatter1 = ax[0,0].scatter(trial_df['embedding_x'],
            trial_df['embedding_y'],
            c=trial_df['trial_indice'].astype(float),
            s=0.1,
            cmap='cividis')
ax[0,0].set_title('Trial: Time from Tone Onset', fontsize= 16)

scatter2 = ax[0,1].scatter(subsample_cluster_df['embedding_x'],
            subsample_cluster_df['embedding_y'],
            c=subsample_cluster_df['Recording'].map(recording_colors),
            label = subsample_cluster_df['Recording'],
            s=0.005)   
ax[0,1].set_title('Recordings', fontsize= 16)
legend2 = ax[0,1].legend(*scatter2.legend_elements(), bbox_to_anchor = (1,1))

scatter3 = ax[0,2].scatter(subsample_cluster_df['embedding_x'],
            subsample_cluster_df['embedding_y'],
            c=subsample_cluster_df['Strain'].map(strain_colors),
            label=subsample_cluster_df['Strain'],
            s=0.005)  
ax[0,2].set_title('Strains', fontsize= 16)
legend_handles3 = [Patch(color=color, label=label) for label, color in strain_colors.items()]
legend3 = ax[0,2].legend(handles = legend_handles3, bbox_to_anchor = (1,1))

scatter4 = ax[1,0].scatter(subsample_cluster_df['embedding_x'],
            subsample_cluster_df['embedding_y'],
            c=subsample_cluster_df['thorax distances'].astype(float),
            s=0.005, cmap = 'afmhot', vmax = 400)  
ax[1,0].set_title('Distance between mice', fontsize= 16)

scatter5 = ax[1,1].scatter(subsample_cluster_df['embedding_x'],
            subsample_cluster_df['embedding_y'],
            c=subsample_cluster_df['distance to port sum'].astype(float),
            s=0.005, cmap = 'afmhot', vmax = 600)  
ax[1,1].set_title('Sum of distance to port', fontsize= 16)

scatter6 = ax[1,2].scatter(subsample_cluster_df['embedding_x'],
            subsample_cluster_df['embedding_y'],
            c=subsample_cluster_df['distance to port diff'].astype(float),
            s=0.005, cmap = 'afmhot', vmax = 350)  
ax[1,2].set_title('Difference in distance to port', fontsize= 16)

scatter7 = ax[2,0].scatter(subsample_cluster_df['embedding_x'],
            subsample_cluster_df['embedding_y'],
            c=subsample_cluster_df['orientation to port sum'].astype(float),
            s=0.005, cmap = 'afmhot', vmax=7)  
ax[2,0].set_title('Sum of orientation to port', fontsize= 16)

scatter8 = ax[2,1].scatter(subsample_cluster_df['embedding_x'],
            subsample_cluster_df['embedding_y'],
            c=subsample_cluster_df['orientation to port diff'].astype(float),
            s=0.005, cmap = 'afmhot', vmax = 4)  
ax[2,1].set_title('Diff in orientation to port', fontsize= 16)

scatter9 = ax[2,2].scatter(subsample_cluster_df['embedding_x'],
            subsample_cluster_df['embedding_y'],
            c=subsample_cluster_df['orientation b/w mice sum'].astype(float),
            s=0.005, cmap = 'afmhot', vmax = 8)  
ax[2,2].set_title('Sum of orientation between mice', fontsize= 16)


scatter10 = ax[3,2].scatter(subsample_cluster_df['embedding_x'],
            subsample_cluster_df['embedding_y'],
            c=subsample_cluster_df['orientation b/w mice diff'].astype(float),
            s=0.005, cmap = 'afmhot', vmax = 4)  
ax[3,2].set_title('Diff in orientation between mice', fontsize= 16)

scatter11 = ax[3,0].scatter(subsample_cluster_df['embedding_x'],
            subsample_cluster_df['embedding_y'],
            c=subsample_cluster_df['mouse velocity sum'].astype(float),
            s=0.005, cmap = 'afmhot', vmax = 20)  
ax[3,0].set_title('Sum of velocities between mice', fontsize= 16)

scatter12 = ax[3,1].scatter(subsample_cluster_df['embedding_x'],
            subsample_cluster_df['embedding_y'],
            c=subsample_cluster_df['mouse velocity diff'].astype(float),
            s=0.005, cmap = 'afmhot', vmax = 12.5)  
ax[3,1].set_title('Diff in velocities between mice', fontsize= 16)


plt.colorbar(scatter1, ax=ax[0,0]) 
plt.colorbar(scatter4, ax=ax[1,0]) 
plt.colorbar(scatter5, ax=ax[1,1]) 
plt.colorbar(scatter6, ax=ax[1,2]) 
plt.colorbar(scatter7, ax=ax[2,0]) 
plt.colorbar(scatter8, ax=ax[2,1]) 
plt.colorbar(scatter9, ax=ax[2,2]) 
plt.colorbar(scatter10, ax=ax[3,2]) 
plt.colorbar(scatter11, ax=ax[3,0]) 
plt.colorbar(scatter12, ax=ax[3,1]) 

#legend_handles4 = [Patch(color=color, label=label) for label, color in strain_colors.items()]
#legend3 = ax[3].legend(handles = legend_handles3)
plt.savefig('./feature_clusters.png')
plt.show()


## Cluster Analysis

### Strain

In [ ]:
cluster_grouping = clustered_df.groupby(['kmeans_label','Strain'])
strain_counts = cluster_grouping['Strain'].value_counts().unstack()
strain_counts.plot(kind='bar', stacked=True, color = ['r','b'],figsize=(10, 6))
plt.xlabel('Cluster')
plt.ylabel('Number of Frames')







In [ ]:
cluster_grouping = clustered_df.groupby(['raw_kmeans_labels','Strain'])
strain_counts = cluster_grouping['Strain'].value_counts().unstack()
strain_counts.plot(kind='bar', stacked=True, color = ['r','b'],figsize=(10, 6))
plt.xlabel('Cluster')
plt.ylabel('Number of Frames')

### Trial


In [ ]:
cluster_grouping_trial = subsample_df.groupby(['kmeans_label','trial_indice'])

trial_counts = cluster_grouping_trial['trial_indice'].value_counts().unstack()
plt.figure()
trial_counts.plot(kind='bar', stacked=True)    
plt.legend([]) 

In [ ]:
def categorize_trial_indices(value):
    value = float(value)
    if value == 0:
        return 'ITI'
    elif 0 < value <= 3:
        return '0-3 sec from tone onset'
    elif 3 < value <= 7:
        return '3-7 sec from tone onset'
    elif 7 < value <= 10:
        return '7-10 sec from tone onset'
    else:
        return None  
    

# Assign values based on the ranges
subsample_df['simplified_trial_indice'] = subsample_df['trial_indice'].apply(lambda x: categorize_trial_indices(x))
trial_colors = make_color_dict('simplified_trial_indice', subsample_df, 'summer')

cluster_grouping_trial = subsample_df.groupby(['kmeans_label','simplified_trial_indice'])

trial_counts = cluster_grouping_trial['simplified_trial_indice'].value_counts().unstack()
trial_counts = trial_counts[['ITI','0-3 sec from tone onset',
 '3-7 sec from tone onset',
 '7-10 sec from tone onset'
 ]]


plt.figure()
trial_counts.plot(kind='bar', stacked=True, color = trial_colors)    
plt.legend()
plt.xlabel('Cluster')
plt.ylabel('Number of Frames')
plt.title('Trial Time per cluster')
plt.ylim((20000.120000))

In [ ]:

def categorize_trial_indices(value):
    value = float(value)
    if value == 0:
        return 'ITI'
    elif 0 < value <= 3:
        return '0-3 sec from tone onset'
    elif 3 < value <= 7:
        return '3-7 sec from tone onset'
    elif 7 < value <= 10:
        return '7-10 sec from tone onset'
    else:
        return None  
    

# Assign values based on the ranges
clustered_df['simplified_trial_indice'] = clustered_df['trial_indice'].apply(lambda x: categorize_trial_indices(x))
trial_colors = make_color_dict('simplified_trial_indice', clustered_df, 'summer')

cluster_grouping_trial = clustered_df.groupby(['raw_kmeans_labels','simplified_trial_indice'])

trial_counts = cluster_grouping_trial['simplified_trial_indice'].value_counts().unstack()
trial_counts = trial_counts[['ITI','0-3 sec from tone onset',
 '3-7 sec from tone onset',
 '7-10 sec from tone onset'
 ]]


plt.figure()
trial_counts.plot(kind='bar', stacked=True, color = trial_colors)    
plt.legend()
plt.xlabel('Cluster')
plt.ylabel('Number of Frames')
plt.title('Trial Time per cluster')


## GIF Creation

In [ ]:
def cluster_gifs(all_cluster_df, cluster_label, gif_length,vid_folder, output_directory):
    no_clusters = len(all_cluster_df[cluster_label].unique())
    i = 1
    for cluster in all_cluster_df[cluster_label].unique():
        print(f'working on {i} out of {no_clusters} cluster gifs')
        cluster_df = all_cluster_df[all_cluster_df[cluster_label] == cluster]
        random_indices = np.random.choice(len(cluster_df), size=gif_length*30, replace=False)
        gif_df = cluster_df.iloc[random_indices]
        #gif_df.to_excel(os.path.join('tester.xlsx', output_directory))
        grouped = gif_df.groupby('Recording')['frame indice'].apply(list).reset_index()
        frames_df = grouped.apply(lambda row: extract_frames(row['Recording'], row['frame indice'], vid_folder), axis = 1)
        create_gif(cluster, frames_df, output_directory)
        i+=1

def find_vid(vid_name, folder):
    ending = ".mp4"
    for filename in os.listdir(folder):
        if filename.endswith(ending) and vid_name in filename:
            video_path = os.path.join(folder, filename)
    return video_path

def extract_frames(vid_name, frame_list, directory):
    # Open the video file
    video_path = find_vid(vid_name, directory)
    cap = cv2.VideoCapture(video_path)
    frames = []
    for frame_index in frame_list:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(float(frame_index))-1)
        success, frame = cap.read()
        if success:
            resized_frame = cv2.resize(frame, (640, 480))
            frame_rgb = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
    cap.release()
    return frames
        
def create_gif(cluster, df, output_directory):
    gif_frames = []
    for recording in range(len(df.index)):
        row = df.loc[recording]
        i = 0
        for frame in row:
            resized_frame = cv2.resize(frame, (640, 480))
            frame_rgb = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
            gif_frames.append(frame_rgb)
            i +=1
    gif_path = os.path.join(output_directory, f"cluster{cluster}.gif")
    imageio.mimsave(gif_path, gif_frames, format='GIF', duration=3)

vid_folder = r'C:\Users\megha\Documents\GitHub\social_dominance_active_inference\data\videos'
output_directory = r'C:\Users\megha\Documents\GitHub\social_dominance_active_inference\results\cluster_gifs_raw'
cluster_gifs(clustered_df, cluster_label = 'raw_kmeans_labels', gif_length = 5, vid_folder=vid_folder, output_directory=output_directory)

   

## Ethogram

In [ ]:
# goal is to create a bar graph for one recording 
file = '10-03-22_Test_23_5-2v5-4'
tester_df = clustered_df[clustered_df['Recording'] == file]
tester_df[['frame indice', 'kmeans_label']]

In [ ]:
def get_indices(df, cluster_label):
    """
    Takes in an indexed key or a list of repeated items, 
    creates a list of indices that correspond to each unique item. 
    
    Args (1):
        repeated_items_list: list, list of repeated items 
    
    Returns:
        item_indices: list of tuples, where the first element is the first index 
                      of an item, and the second element is the last index of that 
                      item 
    """
    is_first = True
    item_indices = {}
    durations = {}
    for i in range(len(df)):
        if is_first:
            current_item = df.iloc[i][cluster_label]
            start_index = df.iloc[i]['frame indice']
            current_recording = df.iloc[i]['Recording']
            start_index = float(start_index)/30/60
            is_first = False
        if df.iloc[i]['Recording'] != current_recording:
            end_index = df.iloc[i-1]['frame indice']
            end_index = float(end_index)/30/60
            indice_list = item_indices[current_item]
            indice_list.append([start_index,end_index])
            durations[current_item].append(end_index - start_index)
            item_indices[current_item] = indice_list
            current_recording = df.iloc[i]['Recording']
            start_index = df.iloc[i]['frame indice']
            start_index = float(start_index)/30/60
        if i == (len(df)-1):
            end_index = df.iloc[i]['frame indice']
            end_index = float(end_index)/30/60
            indice_list = item_indices[current_item]
            indice_list.append([start_index,end_index])
            durations[current_item].append(end_index - start_index)
            item_indices[current_item] = indice_list
        if (not is_first) & (df.iloc[i]['Recording'] == current_recording):
            if df.iloc[i][cluster_label] != current_item:
                end_index = df.iloc[i]['frame indice']
                stop_index = df.iloc[i-1]['frame indice']
                stop_index = float(stop_index)/30/60
                end_index = float(end_index)/30/60
                if current_item in item_indices.keys():
                    indice_list = item_indices[current_item]
                    indice_list.append([start_index,stop_index])
                    durations[current_item].append(end_index - start_index)
                    item_indices[current_item] = indice_list
                else: 
                    item_indices[current_item] = [[start_index, stop_index]]
                    durations[current_item] = [end_index - start_index]
                start_index = df.iloc[i]['frame indice']
                start_index = float(start_index)/30/60
                current_item = df.iloc[i][cluster_label] 
    return item_indices, durations

cluster_dict, durations = get_indices(clustered_df, 'kmeans_label')

recording_cluster_dict, tester_durations = get_indices(tester_df, 'kmeans_label')

  

In [ ]:
avg_durations = {}
for cluster, duration in tester_durations.items():
    duration_min = np.array(duration)*60
    avg_duration = np.mean(duration_min)
    med_durations = np.median(duration_min)
    max_beh = np.max(duration_min)
    min_beh = np.min(duration_min)
    no_events = len(duration)
    avg_durations[cluster] = [avg_duration, med_durations, max_beh, min_beh, no_events]
    

tester_cluster_info = pd.DataFrame.from_dict(avg_durations, orient = 'index')
tester_cluster_info.columns = ['Average Duration (s)','Median (s)', 'Max (s)', 'Min(s)', 'Number of events']
tester_cluster_info

In [ ]:
cluster_colors = make_color_dict('kmeans_label', tester_df, cmap='Spectral')

In [ ]:
avg_durations = {}
for cluster, duration in durations.items():
    duration_min = np.array(duration)*60
    avg_duration = np.mean(duration_min)
    med_durations = np.median(duration_min)
    max_beh = np.max(duration_min)
    min_beh = np.min(duration_min)
    no_events = len(duration)
    avg_durations[cluster] = [avg_duration, med_durations, max_beh]#, min_beh]#, no_events]
    

cluster_info = pd.DataFrame.from_dict(avg_durations, orient = 'index')
cluster_info.columns = ['Average Duration (s)','Median (s)', 'Max (s)'] #'Min(s)']#, 'Number of events']
cluster_info

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (10,5), sharex=True, facecolor='white')
fig.subplots_adjust(hspace=0.05) 
import matplotlib.patches as mpatches

colors = list(cluster_colors.values())
fills = ['/','','.']*10
# Loop through each cluster (index) and plot the bars
for i, cluster in enumerate(cluster_info.index):
    cluster_data = cluster_info.loc[cluster]
    bar_width = 0.3
    x_positions = [i + bar_width * j for j in range(len(cluster_data))]
    for j, (x, value) in enumerate(zip(x_positions, cluster_data)):
        hatch_pattern = fills[j % len(fills)]  # Use modulo to cycle through fill patterns
        # Plot on the first subplot (0-1 on y-axis)
        ax1.bar(x, value, width=bar_width, color=cluster_colors[cluster], hatch=hatch_pattern, edgecolor='white')
        # Plot on the second subplot (0-1 on y-axis)
        ax2.bar(x, value, width=bar_width, color=cluster_colors[cluster], hatch=hatch_pattern, edgecolor='white')
        if j == len(x_positions) // 2:  # Assuming the middle bar is at index len(x_positions) // 2
            ax1.text(x, -2, f'{cluster}', ha='center', va='center')
            ax2.text(x, -0.1, f'{cluster}', ha='center', va='center')
    #ax1.bar(x_positions, cluster_data, width=bar_width, label=f'Cluster {cluster}', color=colors[i], hatch = fills)
    #ax2.bar(x_positions, cluster_data, width=bar_width, label=f'Cluster {cluster}', color=colors[i], hatch = fills)
# ax2.set_xticks([j + 1.5 * bar_width * i for i in range(len(cluster_info.columns)) for j in range(len(cluster_info))])
# ax2.set_xticklabels([col for col in cluster_info.columns for _ in range(len(cluster_info))], rotation=45, ha='right')
ax1.set_ylim(6, 50)  # outliers only
ax2.set_ylim(0, 1.75)  # most of the data

# hide the spines between ax and ax2
 # don't put tick labels at the top

d = .5  # proportion of vertical to horizontal extent of the slanted line
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
              linestyle="none", color='k', mec='k', mew=1, clip_on=False)
ax1.plot([0, 1], [0, 0], transform=ax1.transAxes, **kwargs)
ax2.plot([0, 1], [1, 1], transform=ax2.transAxes, **kwargs)
# Set labels and title


ax2.set_ylabel('Seconds')
ax1.set_title('Cluster Statistics', fontsize = 20)

# Add legend

cluster_labels = []
for cluster, color in cluster_colors.items():
    cluster_labels.append(mpatches.Patch(facecolor=color,  label=cluster))
                          
#ax1.legend(handles = cluster_labels, bbox_to_anchor=(.9,-1.1), ncol=10)                   
patch_max = mpatches.Patch(facecolor='gray', hatch='..', label='Max (s)', edgecolor='white')
patch_avg = mpatches.Patch(facecolor='gray', hatch='//', label='Avg Duration (s)',edgecolor='white')
patch_median = mpatches.Patch(facecolor='gray', hatch='', label='Median (s)',edgecolor='white')

# Plotting empty bars to display the legend
ax1.spines['bottom'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax1.tick_params(labelbottom=False, bottom=False)
ax2.tick_params(labelbottom=False, bottom=False)   


# Create the legend
ax2.legend(handles=[ patch_avg, patch_median, patch_max], bbox_to_anchor=(.75,-.2), ncol= 3, handleheight=1.5)
ax1.grid(False)
ax2.grid(False)

# Show the plot
plt.show()



In [ ]:
fig, axs = plt.subplots(figsize=(10, 2), nrows=1, ncols=1, sharex=True)   
y_labels = list(recording_cluster_dict.keys())
max_len = len(y_labels)
bars = {}
par1 = 1
i = 0
bar_height = 1
for cluster_label in y_labels:
    start_stops = cluster_dict[cluster_label]
    starts = np.array([float(x[0]) for x in start_stops])
    stops = np.array([float(x[1]) for x in start_stops])
    bars[cluster_label] = [starts,stops]
# color
    bar_color = cluster_colors[cluster_label]
    axs.barh(
    ,
    stops - starts,
    left=starts,
    height=bar_height,
    align="center",
    edgecolor=bar_color,
    color=bar_color,
    alpha=1,
    label=cluster_label
    )
    i += 1
#axs.set_ylim(bottom=0, top=(max_len * par1) + par1)
pos = np.arange(par1, max_len * par1 + par1 + 1, par1)
#axs.set_yticks(pos[: len(y_labels)])
axs.set_yticklabels("")
#axs.set_xlim((0,1))
#axs.set_ylabel("Cluster")
axs.grid(color="g", linestyle=":")
axs.set_xlabel("Minute")
axs.invert_yaxis()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.5), fancybox=True, shadow=True, ncol=10)
plt.tight_layout()
plt.show()

In [ ]:
states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
num_states = len(states)
# Example sequence of transitions
transitions = np.array(clustered_df['kmeans_label'])
transitions = transitions[np.append([True], np.diff(transitions) != 0)]
# Create a transition matrix
transition_matrix = np.zeros((len(states), len(states)))

# Populate the transition matrix
for i in range(len(transitions) - 1):
    current_state = transitions[i]
    next_state = transitions[i + 1]
    transition_matrix[current_state][next_state] += 1

# Normalize to get probabilities
transition_matrix = transition_matrix / transition_matrix.sum(axis=1, keepdims=True)

# Print the transition matrix
plt.imshow(transition_matrix, cmap='viridis', interpolation='nearest')

# Adding colorbar
plt.colorbar()

# Adding labels and title
plt.xlabel('Next Cluster')
plt.ylabel('Current Cluster')
plt.title('Transition Matrix Heatmap')
plt.xticks(np.arange(num_states), np.arange(num_states))

# Set y-axis ticks and labels
plt.yticks(np.arange(num_states), np.arange(num_states))
# Show the plot
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(10, 2), nrows=10, ncols=1, sharex=True)
y_labels = list(recording_cluster_dict.keys())
max_len = len(y_labels)
bars = {}
par1 = 1
i = 0
bar_height = 1

for cluster_label in y_labels:
    start_stops = recording_cluster_dict[cluster_label]
    starts = np.array([float(x[0]) for x in start_stops])
    stops = np.array([float(x[1]) for x in start_stops])
    bars[cluster_label] = [starts, stops]

    # color
    bar_color = cluster_colors[cluster_label]
    axs[i].barh(
        0,
        stops - starts,
        left=starts,
        height=bar_height,
        align="center",
        edgecolor=bar_color,
        color=bar_color,
        alpha=1,
        label=cluster_label
    )
    axs[i].set_xticklabels('')
    axs[i].set_yticklabels('')
    axs[i].set_xlim((0,1))
    i += 1

axs[0].set_ylim(bottom=0, top=(max_len * par1) + par1)
pos = np.arange(par1, max_len * par1 + par1 + 1, par1)

# Adjust y-axis ticks and labels
#axs[0].set_yticks(pos[: len(y_labels)])
#axs[0].set_yticklabels(y_labels)  # Use the cluster labels as y-axis tick labels

# axs[0].set_ylabel("Cluster")
# axs[0].grid(color="g", linestyle=":")
# axs[0].set_xlabel("Minute")
# axs[0].invert_yaxis()


plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.5), fancybox=True, shadow=True, ncol=10)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3))
y_labels = list(recording_cluster_dict.keys())
max_len = len(y_labels)
bars = {}
par1 = 1
bar_height = .75  # Adjust the bar height as needed

for i, cluster_label in enumerate(y_labels):
    start_stops = recording_cluster_dict[cluster_label]
    starts = np.array([float(x[0]) for x in start_stops])
    stops = np.array([float(x[1]) for x in start_stops])
    bars[cluster_label] = [starts, stops]

    # color
    bar_color = cluster_colors[cluster_label]
    ax.barh(
        1,
        stops - starts,
        left=starts,
        height=bar_height,
        align="center",
        edgecolor=bar_color,
        color=bar_color,
        alpha=1,
        label=cluster_label
    )

#ax.set_yticks(np.arange(len(y_labels)))
ax.set_yticklabels("")
ax.set_xlabel("First Three Minutes", fontsize = 14)
ax.set_title("Ethogram", fontsize = 16)
ax.invert_yaxis()
ax.barh(1, .167, left = 1.14, height=bar_height, align='center', edgecolor='black', color='none', linewidth=3, label = 'Tone')
ax.barh(1, .167, left = 2.48, height=bar_height, align='center', edgecolor='black', color='none', linewidth=3)
#1.14277778,  2.47611111
ax.set_xlim((0,3))
ax.tick_params(left=False)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.5), fancybox=True, shadow=True, ncol=11)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3))
y_labels = list(recording_cluster_dict.keys())
max_len = len(y_labels)
bars = {}
par1 = 1
bar_height = .75  # Adjust the bar height as needed

for i, cluster_label in enumerate(y_labels):
    start_stops = recording_cluster_dict[cluster_label]
    starts = np.array([float(x[0]) for x in start_stops])
    stops = np.array([float(x[1]) for x in start_stops])
    bars[cluster_label] = [starts, stops]

    # color
    bar_color = cluster_colors[cluster_label]
    ax.barh(
        1,
        stops - starts,
        left=starts,
        height=bar_height,
        align="center",
        edgecolor=bar_color,
        color=bar_color,
        alpha=1,
        label=cluster_label
    )

#ax.set_yticks(np.arange(len(y_labels)))
ax.set_yticklabels("")
ax.set_xlabel("25th Minute", fontsize = 14)
ax.set_title("Ethogram", fontsize = 16)
ax.invert_yaxis()
ax.barh(1, .167, left = 25.31, height=bar_height, align='center', edgecolor='black', color='none', linewidth=3, label = 'Tone')
ax.set_xlim((25,26))
ax.tick_params(left=False)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.5), fancybox=True, shadow=True, ncol=11)
plt.tight_layout()
plt.show()

In [ ]:
tester_tones = tone_times_df[tone_times_df['File Name'] ==file]
tester_tones

In [ ]:
tester_tones = list(tester_tones['all_tone_frame'])
tester_tones = tester_tones[0]
tester_tones = np.array(tester_tones) / 30 / 60


In [ ]:
tester_tones

### Raw clustering Stats


In [ ]:
# goal is to create a bar graph for one recording 
file = '10-03-22_Test_23_5-2v5-4'
tester_df = clustered_df[clustered_df['Recording'] == file]
tester_df[['frame indice', 'raw_kmeans_labels']]

In [ ]:
raw_cluster_dict, raw_durations = get_indices(clustered_df, 'raw_kmeans_labels')

raw_recording_cluster_dict, raw_tester_durations = get_indices(tester_df, 'raw_kmeans_labels')

In [ ]:
avg_durations = {}
for cluster, duration in raw_durations.items():
    duration_min = np.array(duration)*60
    avg_duration = np.mean(duration_min)
    med_durations = np.median(duration_min)
    max_beh = np.max(duration_min)
    min_beh = np.min(duration_min)
    no_events = len(duration)
    avg_durations[cluster] = [avg_duration, med_durations, max_beh]#, min_beh]#, no_events]
    

cluster_info = pd.DataFrame.from_dict(avg_durations, orient = 'index')
cluster_info.columns = ['Average Duration (s)','Median (s)', 'Max (s)'] #'Min(s)']#, 'Number of events']
cluster_info

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (10,5), sharex=True, facecolor='white')
fig.subplots_adjust(hspace=0.05) 
import matplotlib.patches as mpatches

colors = list(cluster_colors.values())
fills = ['/','','.']*10
# Loop through each cluster (index) and plot the bars
for i, cluster in enumerate(cluster_info.index):
    cluster_data = cluster_info.loc[cluster]
    bar_width = 0.3
    x_positions = [i + bar_width * j for j in range(len(cluster_data))]
    for j, (x, value) in enumerate(zip(x_positions, cluster_data)):
        hatch_pattern = fills[j % len(fills)]  # Use modulo to cycle through fill patterns
        # Plot on the first subplot (0-1 on y-axis)
        ax1.bar(x, value, width=bar_width, color=cluster_colors[cluster], hatch=hatch_pattern, edgecolor='white')
        # Plot on the second subplot (0-1 on y-axis)
        ax2.bar(x, value, width=bar_width, color=cluster_colors[cluster], hatch=hatch_pattern, edgecolor='white')
        if j == len(x_positions) // 2:  # Assuming the middle bar is at index len(x_positions) // 2
            ax1.text(x, -2, f'{cluster}', ha='center', va='center')
            ax2.text(x, -0.1, f'{cluster}', ha='center', va='center')
    #ax1.bar(x_positions, cluster_data, width=bar_width, label=f'Cluster {cluster}', color=colors[i], hatch = fills)
    #ax2.bar(x_positions, cluster_data, width=bar_width, label=f'Cluster {cluster}', color=colors[i], hatch = fills)
# ax2.set_xticks([j + 1.5 * bar_width * i for i in range(len(cluster_info.columns)) for j in range(len(cluster_info))])
# ax2.set_xticklabels([col for col in cluster_info.columns for _ in range(len(cluster_info))], rotation=45, ha='right')
ax1.set_ylim(6, 70)  # outliers only
ax2.set_ylim(0, 1.75)  # most of the data

# hide the spines between ax and ax2
 # don't put tick labels at the top

d = .5  # proportion of vertical to horizontal extent of the slanted line
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
              linestyle="none", color='k', mec='k', mew=1, clip_on=False)
ax1.plot([0, 1], [0, 0], transform=ax1.transAxes, **kwargs)
ax2.plot([0, 1], [1, 1], transform=ax2.transAxes, **kwargs)
# Set labels and title


ax2.set_ylabel('Seconds')
ax1.set_title('Cluster Statistics', fontsize = 20)

# Add legend

cluster_labels = []
for cluster, color in cluster_colors.items():
    cluster_labels.append(mpatches.Patch(facecolor=color,  label=cluster))
                          
#ax1.legend(handles = cluster_labels, bbox_to_anchor=(.9,-1.1), ncol=10)                   
patch_max = mpatches.Patch(facecolor='gray', hatch='..', label='Max (s)', edgecolor='white')
patch_avg = mpatches.Patch(facecolor='gray', hatch='//', label='Avg Duration (s)',edgecolor='white')
patch_median = mpatches.Patch(facecolor='gray', hatch='', label='Median (s)',edgecolor='white')

# Plotting empty bars to display the legend
ax1.spines['bottom'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax1.tick_params(labelbottom=False, bottom=False)
ax2.tick_params(labelbottom=False, bottom=False)   


# Create the legend
ax2.legend(handles=[ patch_avg, patch_median, patch_max], bbox_to_anchor=(.75,-.2), ncol= 3, handleheight=1.5)
ax1.grid(False)
ax2.grid(False)

# Show the plot
plt.show()


In [ ]:
states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
num_states = len(states)
# Example sequence of transitions
transitions = np.array(clustered_df['raw_kmeans_labels'])
transitions = transitions[np.append([True], np.diff(transitions) != 0)]
# Create a transition matrix
transition_matrix = np.zeros((len(states), len(states)))

# Populate the transition matrix
for i in range(len(transitions) - 1):
    current_state = transitions[i]
    next_state = transitions[i + 1]
    transition_matrix[current_state][next_state] += 1

# Normalize to get probabilities
transition_matrix = transition_matrix / transition_matrix.sum(axis=1, keepdims=True)

# Print the transition matrix
plt.imshow(transition_matrix, cmap='viridis', interpolation='nearest')

# Adding colorbar
plt.colorbar()

# Adding labels and title
plt.xlabel('Next Cluster')
plt.ylabel('Current Cluster')
plt.title('Transition Matrix Heatmap')
plt.xticks(np.arange(num_states), np.arange(num_states))

# Set y-axis ticks and labels
plt.yticks(np.arange(num_states), np.arange(num_states))
# Show the plot
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3))
y_labels = list(raw_recording_cluster_dict.keys())
max_len = len(y_labels)
bars = {}
par1 = 1
bar_height = .75  # Adjust the bar height as needed

for i, cluster_label in enumerate(y_labels):
    start_stops = raw_recording_cluster_dict[cluster_label]
    starts = np.array([float(x[0]) for x in start_stops])
    stops = np.array([float(x[1]) for x in start_stops])
    bars[cluster_label] = [starts, stops]

    # color
    bar_color = cluster_colors[cluster_label]
    ax.barh(
        1,
        stops - starts,
        left=starts,
        height=bar_height,
        align="center",
        edgecolor=bar_color,
        color=bar_color,
        alpha=1,
        label=cluster_label
    )

#ax.set_yticks(np.arange(len(y_labels)))
ax.set_yticklabels("")
ax.set_xlabel("First Three Minutes", fontsize = 14)
ax.set_title("Ethogram", fontsize = 16)
ax.invert_yaxis()
ax.barh(1, .167, left = 1.14, height=bar_height, align='center', edgecolor='black', color='none', linewidth=3, label = 'Tone')
ax.barh(1, .167, left = 2.48, height=bar_height, align='center', edgecolor='black', color='none', linewidth=3)
#1.14277778,  2.47611111
ax.set_xlim((0,3))
ax.tick_params(left=False)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.5), fancybox=True, shadow=True, ncol=11)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3))
y_labels = list(raw_recording_cluster_dict.keys())
max_len = len(y_labels)
bars = {}
par1 = 1
bar_height = .75  # Adjust the bar height as needed

for i, cluster_label in enumerate(y_labels):
    start_stops = raw_recording_cluster_dict[cluster_label]
    starts = np.array([float(x[0]) for x in start_stops])
    stops = np.array([float(x[1]) for x in start_stops])
    bars[cluster_label] = [starts, stops]

    # color
    bar_color = cluster_colors[cluster_label]
    ax.barh(
        1,
        stops - starts,
        left=starts,
        height=bar_height,
        align="center",
        edgecolor=bar_color,
        color=bar_color,
        alpha=1,
        label=cluster_label
    )

#ax.set_yticks(np.arange(len(y_labels)))
ax.set_yticklabels("")
ax.set_xlabel("25th Minute", fontsize = 14)
ax.set_title("Ethogram", fontsize = 16)
ax.invert_yaxis()
ax.barh(1, .167, left = 25.31, height=bar_height, align='center', edgecolor='black', color='none', linewidth=3, label = 'Tone')
ax.set_xlim((25,26))
ax.tick_params(left=False)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.5), fancybox=True, shadow=True, ncol=11)
plt.tight_layout()
plt.show()